The following code solves a 5-dimensional PDF transport equation of turbulent flows using **TT-CUR-DEIM** algorithm. TT-CUR-DEIM is our proposed method for solving high-dimensional nonlinear tensor differential equations on **tensor-train** low-rank manifolds. This algorithm is presented in the following paper.

**Cross interpolation for solving high-dimensional dynamical systems on low-rank Tucker and tensor train manifolds**      [https://www.sciencedirect.com/science/article/pii/S0045782524006406]


<br>
The following PDE is discretized using the finite difference method in physical and species domains to obtain a tensor differential equation. We use the explicit fourth-order Runge-Kutta time integration scheme for time advancement.

$$
\frac{\partial p}{\partial t}
+ u_i(x_1, x_2, x_3)\,\frac{\partial p}{\partial x_i}
=
\frac{\partial}{\partial x_i}
\left[
\nu(x_1, x_2, x_3)\,\frac{\partial p}{\partial x_i}
\right]
+
\frac{\partial}{\partial \psi_\alpha}
\left[
\Omega(x_1, x_2, x_3)\,(\psi_\alpha - \overline{\Psi}_\alpha)\,p
\right]
-
\frac{\partial}{\partial \psi_\alpha}
\left[
S_\alpha(\psi_1, \dots, \psi_{n_s})\,p
\right]
$$


<br>
For the velocity field, we use the ABC method. The parameters used in the code are as follows: 

* Velocity field
$$
u_1 = A \sin(k x_3) + C \cos(k x_2)
$$
$$
u_2 = B \sin(k x_1) + A \cos(k x_3)
$$
$$
u_3 = C \sin(k x_2) + B \cos(k x_1)
$$

<br>

* Parameters
$$
\nu(x_1, x_2, x_3) = \nu_L + \nu_t(x_1, x_2, x_3)
$$
$$
\nu_t(x_1, x_2, x_3) = C_s \Delta^2 |S|
$$
$$
\Omega(x_1, x_2, x_3) = C_\omega \frac{\nu(x_1, x_2, x_3)}{\Delta^2}
$$

<br>

* Linear reaction
$$
S_1 = -k_1 \psi_1
$$
$$
S_2 = k_2 \psi_1
$$

<br>

* Linear reaction — analytical solution
$$
\overline{\Psi}_1(t) = \overline{\Psi}_1(0)\,\exp(-k_1 t)
$$
$$
\overline{\Psi}_2(t) = \overline{\Psi}_2(0)
+ \frac{k_2}{k_1}\,\overline{\Psi}_1(0)\,[1 - \exp(-k_1 t)]
$$

<br>

* Nonlinear reaction
$$
S_1 = -D_\alpha \psi_1 \psi_2
$$
$$
S_2 = -C_r D_\alpha \psi_1 \psi_2
$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools as it

# ============================================================================================================
# Discretizition parameters-------------------

dim = 5
X_min, X_max = 0, 2*np.pi
nx = 90 # Number of points in physical domain
Nx = np.tile(nx, 3)
Xp = (np.linspace(X_min, X_max, nx, endpoint=False)).reshape(-1,1)

Delta_X = (X_max-X_min)/(nx)
D1x = np.diag(np.ones(nx-1), k=1) - np.diag(np.ones(nx-1), k=-1)
D1x[-1,0]= 1.0 ; D1x[0,-1]= -1.0
D1x = D1x/(2*Delta_X)

D2x = -2*np.eye(nx) + np.diag(np.ones(nx-1), k=1) + np.diag(np.ones(nx-1), k=-1)
D2x[-1,0]=1.0;  D2x[0,-1]=1.0
D2x = D2x/(Delta_X**2)

Phi_min, Phi_max = 0, 1
ns = 120  # Number of points in species domain
Ns = np.tile(ns, dim-3)
N = np.hstack((Nx,Ns))

Phi = (np.linspace(Phi_min, Phi_max, ns)).reshape(-1,1)
Delta_Phi = Phi[3]-Phi[2]
D1_Phi = np.diag(np.ones(ns-1), k=1) - np.diag(np.ones(ns-1), k=-1)
D1_Phi[0,0] = -1.0 ;   D1_Phi[-1,-1] = 1.0
D1_Phi[1:-1] = D1_Phi[1:-1]/(2*Delta_Phi)
D1_Phi[0,:] = D1_Phi[0,:]/(Delta_Phi)
D1_Phi[-1,:] = D1_Phi[-1,:]/(Delta_Phi)

D2_Phi = -2*np.eye(ns) + np.diag(np.ones(ns-1), k=1) + np.diag(np.ones(ns-1), k=-1)
D2_Phi = D2_Phi/(Delta_Phi**2)

All_D2 = []
[All_D2.append(D2x) for i in range(3)]
[All_D2.append(D2_Phi) for i in range(dim-3)]


# ============================================================================================================
# ROM parameters -----------------------------
dt = 4 * 10 ** (-4)  # Time step
Total_time = 5 # Total time
r = np.tile(10, dim-1)

N_os = 5
eps_U = 10**-6
eps_L = 10**-8

np.random.seed(20)

max_iteration = 35 # Maximum number of iteration for TT-DEIM on initial condition

# Turbulence parameters --------------------
Nu_L = 0.0005
Cs = 0.1
Cw = 0.5
D_alpha = 0.1
r_S = 1

# Define velocity field with Arnold-Beltrami-Childress method and compute norm of the strain rate tensor -------------------
A=0.1;   B=0.1;   C=0.1*np.sqrt(2);   Kk=2   # ABC method
# A=1;   B=1;   C=-2;   a=1;   b=1;   c=1    # Taylor Green method

vel = []
v1 = np.zeros((Nx[0], Nx[1], Nx[2])); v2 = np.zeros((Nx[0], Nx[1], Nx[2])); v3 = np.zeros((Nx[0], Nx[1], Nx[2]));
Strain_norm = np.zeros((Nx[0], Nx[1], Nx[2]))

for i, j, h in it.product(range(Nx[0]), range(Nx[1]), range(Nx[2])):
    # ABC method
    v1[i,j,h] = np.squeeze(A*np.sin(Kk*Xp[h])+C*np.cos(Kk*Xp[j]))
    v2[i,j,h] = np.squeeze(B*np.sin(Kk*Xp[i])+A*np.cos(Kk*Xp[h]))
    v3[i,j,h] = np.squeeze(C*np.sin(Kk*Xp[j])+B*np.cos(Kk*Xp[i]))

    Strain_12 = np.squeeze(0.5*(-C*Kk*np.sin(Kk*Xp[j]) + B*Kk*np.cos(Kk*Xp[i]))) # S12 = S21 = 1/2 (-Cksin(kx2) + Bkcos(kx1))
    Strain_13 = np.squeeze(0.5*( A*Kk*np.cos(Kk*Xp[h]) - B*Kk*np.sin(Kk*Xp[i]))) # S13 = S31 = 1/2 (Akcos(kx3) - Bksin(kx1))
    Strain_23 = np.squeeze(0.5*(-A*Kk*np.sin(Kk*Xp[h]) + C*Kk*np.cos(Kk*Xp[j]))) # S23 = S32 = 1/2 (-Aksin(kx3) + Ckcos(kx2))

    Strain_tensor = np.array([[0, Strain_12, Strain_13], [Strain_12, 0, Strain_23], [Strain_13, Strain_23, 0]])

    
    # Taylor Green method
    # v1[i,j,h] = A*np.cos(a*Xp[i]) * np.sin(b*Xp[j]) * np.sin(c*Xp[h])
    # v2[i,j,h] = B*np.sin(a*Xp[i]) * np.cos(b*Xp[j]) * np.sin(c*Xp[h])
    # v3[i,j,h] = C*np.sin(a*Xp[i]) * np.sin(b*Xp[j]) * np.cos(c*Xp[h])

    # Strain_11 = np.squeeze(-A*a*np.sin(a*Xp[i])*np.sin(b*Xp[j])*np.sin(c*Xp[h]))
    # Strain_12 = np.squeeze(0.5*(A*b + B*a)*np.cos(a*Xp[i])*np.cos(b*Xp[j])*np.sin(c*Xp[h]))
    # Strain_13 = np.squeeze(0.5*(A*c + C*a)*np.cos(a*Xp[i])*np.sin(b*Xp[j])*np.cos(c*Xp[h]))
    # Strain_22 = np.squeeze(-B*b*np.sin(a*Xp[i])*np.sin(b*Xp[j])*np.sin(c*Xp[h]))
    # Strain_23 = np.squeeze(0.5*(B*c + C*b)*np.sin(a*Xp[i])*np.cos(b*Xp[j])*np.cos(c*Xp[h]))
    # Strain_33 = np.squeeze(-C*c*np.sin(a*Xp[i])*np.sin(b*Xp[j])*np.sin(c*Xp[h]))

    # Strain_tensor = np.array([[Strain_11, Strain_12, Strain_13], [Strain_12, Strain_22, Strain_23], [Strain_13, Strain_23, Strain_33]])


    Strain_norm[i,j,h] = np.linalg.norm(Strain_tensor)


vel.append(v1);  vel.append(v2);  vel.append(v3)

# --------------------------------------------
Nu_x = Cs * Delta_X**2 * Strain_norm
Nu = Nu_L + Nu_x
Omega = (Cw*Nu)/(Delta_X**2)
S_alpha = np.hstack((-0.1*Phi.reshape(-1,1), 0.05*Phi.reshape(-1,1)))

Nu_derivation = []
for jj in range(3):
    Nu_der = (D1x @  np.swapaxes(Nu, jj, 0).reshape(N[jj], -1)).reshape(N[0], N[1], N[2])  # This computes ∂Nu/∂x_i 
    Nu_der = np.swapaxes(Nu_der, jj, 0)  # To get the original axis order of Nu
    Nu_derivation.append(Nu_der)


# ============================================================================================================
# Initial Condition
IC_mu_A = [0.4, 0.6]
IC_cov_A = np.matrix([[0.012, 0], [0, 0.012]])

IC_mu_B = [0.6, 0.4]
IC_cov_B = np.matrix([[0.012, 0], [0, 0.012]])

IC_mu = np.zeros((Nx[0], 2))
IC_Cf = np.zeros((Nx[0], 2))
for i in range(Nx[0]):
  if 29 < i < 59:
    IC_mu[i,:] = IC_mu_B
    IC_Cf[i,:] = [0, 1]
  else:
    IC_mu[i,:] = IC_mu_A
    IC_Cf[i,:] = [1, 0]


IC_mu_soft = IC_mu.copy()
IC_Cf_soft = IC_Cf.copy()
for i in range(10,len(Xp)-10):
  IC_mu_soft[i,:] = np.sum(IC_mu[i-3:i+4, :], axis=0)/7
  IC_Cf_soft[i,:] = np.sum(IC_Cf[i-3:i+4, :], axis=0)/7


# ============================================================================================================'
#  DEIM Algorithm
def myDEIM(U, r):
  n = U.shape[0]
  z = np.zeros((n,1))
  max_idx = np.argmax(abs(U[:,0]))
  U_m = np.copy(U[:,0:1])
  DEIM_points = np.array([max_idx])
  P = np.copy(z).reshape(-1,1)
  P[max_idx, 0] = 1

  for j in range(1,r):
      c = np.linalg.inv(P.T@U_m) @ (P.T@U[:, j:j+1])
      Res = U[:, j:j+1] - U_m@c
      max_idx = np.argmax(abs(Res))
      U_m = np.hstack((U_m, U[:,j:j+1]))
      DEIM_points = np.append(DEIM_points, max_idx)
      P = np.hstack((P, z))
      P[max_idx, j] = 1
  return DEIM_points

# ============================================================================================================
def GappyPOD(U, N_os, DEIM_idx):
  P = np.copy(DEIM_idx)
  for i in range(N_os):
    _,S,V = np.linalg.svd(U[P.reshape(-1,1), :], full_matrices=False)  
    g = S[-2]**2 - S[-1]**2
    Ub = V @ U.T
    z = g + np.sum(Ub**2, axis=0)
    z = z - np.sqrt((g+np.sum(Ub**2, axis=0))**2 - 4*g*Ub[-1, :]**2)
    Ii = np.flip(np.argsort(z)).squeeze()
    e = 0
    while len(np.intersect1d(P , Ii[e])) > 0:
      e += 1
    P = np.hstack((P, Ii[e]))
  
  return P

# ============================================================================================================
# Tensor Train function

def eval_input(indx, k, mode, d1, d2, d3):

    if np.remainder(k,2)==0:   # Performing Forward Iteration
        
        if mode == 0:
            f = np.zeros((d1, d2, d3))
            for r_l, ni, r_r in it.product(range(d1), range(d2), range(d3)):
                X_minus_MU_A, X_minus_MU_B = [], []
                for ii in range(3,dim):
                    x_minus_mu_A =  Phi[indx[ii][r_r]] - IC_mu_A[ii-3]
                    x_minus_mu_B =  Phi[indx[ii][r_r]] - IC_mu_B[ii-3]
                    X_minus_MU_A.append(x_minus_mu_A)
                    X_minus_MU_B.append(x_minus_mu_B)

                X_minus_MU_A = np.array(X_minus_MU_A).reshape(-1,1)
                X_minus_MU_B = np.array(X_minus_MU_B).reshape(-1,1)

                f[r_l, ni, r_r] += IC_Cf_soft[indx[0][ni], 0] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_B))) * np.exp( -0.5*(X_minus_MU_B).T @ np.linalg.inv(IC_cov_B) @ (X_minus_MU_B) ) \
                    + IC_Cf_soft[indx[0][ni], 1] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_A))) * np.exp( -0.5*(X_minus_MU_A).T @ np.linalg.inv(IC_cov_A) @ (X_minus_MU_A) )
        
        if 0 < mode < 3:
            f = np.zeros((d1, d2, d3))
            for r_l, ni, r_r in it.product(range(d1), range(d2), range(d3)):
                X_minus_MU_A, X_minus_MU_B = [], []
                for ii in range(3,dim):
                    x_minus_mu_A =  Phi[indx[ii][r_r]] - IC_mu_A[ii-3]
                    x_minus_mu_B =  Phi[indx[ii][r_r]] - IC_mu_B[ii-3]
                    X_minus_MU_A.append(x_minus_mu_A)
                    X_minus_MU_B.append(x_minus_mu_B)

                X_minus_MU_A = np.array(X_minus_MU_A).reshape(-1,1)
                X_minus_MU_B = np.array(X_minus_MU_B).reshape(-1,1)

                f[r_l, ni, r_r] += IC_Cf_soft[indx[0][r_l], 0] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_B))) * np.exp( -0.5*(X_minus_MU_B).T @ np.linalg.inv(IC_cov_B) @ (X_minus_MU_B) ) \
                    + IC_Cf_soft[indx[0][r_l], 1] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_A))) * np.exp( -0.5*(X_minus_MU_A).T @ np.linalg.inv(IC_cov_A) @ (X_minus_MU_A) )

        if mode > 2:
            f = np.zeros((d1, d2, d3))
            for r_l, ni, r_r in it.product(range(d1), range(d2), range(d3)):
                X_minus_MU_A, X_minus_MU_B = [], []
                for ii in range(3,mode):
                    x_minus_mu_A =  Phi[indx[ii][r_l]] - IC_mu_A[ii-3]
                    x_minus_mu_B =  Phi[indx[ii][r_l]] - IC_mu_B[ii-3]
                    X_minus_MU_A.append(x_minus_mu_A)
                    X_minus_MU_B.append(x_minus_mu_B)

                X_minus_MU_A.append(Phi[indx[mode][ni]] - IC_mu_A[mode-3])
                X_minus_MU_B.append(Phi[indx[mode][ni]] - IC_mu_B[mode-3])

                for ii in range(mode+1,dim):
                    x_minus_mu_A =  Phi[indx[ii][r_r]] - IC_mu_A[ii-3]
                    x_minus_mu_B =  Phi[indx[ii][r_r]] - IC_mu_B[ii-3]
                    X_minus_MU_A.append(x_minus_mu_A)
                    X_minus_MU_B.append(x_minus_mu_B)

                X_minus_MU_A = np.array(X_minus_MU_A).reshape(-1,1)
                X_minus_MU_B = np.array(X_minus_MU_B).reshape(-1,1)

                f[r_l, ni, r_r] += IC_Cf_soft[indx[0][r_l], 0] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_B))) * np.exp( -0.5*(X_minus_MU_B).T @ np.linalg.inv(IC_cov_B) @ (X_minus_MU_B) )\
                    + IC_Cf_soft[indx[0][r_l], 1] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_A))) * np.exp( -0.5*(X_minus_MU_A).T @ np.linalg.inv(IC_cov_A) @ (X_minus_MU_A) )

    
    if np.remainder(k,2)==1:   # Performing Reverse Iteration
        
        if mode == dim-1:
            f = np.zeros((d1, d2, d3))
            for r_l, ni, r_r in it.product(range(d1), range(d2), range(d3)):
                X_minus_MU_A, X_minus_MU_B = [], []
                for ii in range(0, dim-3):
                    x_minus_mu_A =  Phi[indx[ii][r_l]] - IC_mu_A[ii]
                    x_minus_mu_B =  Phi[indx[ii][r_l]] - IC_mu_B[ii]
                    X_minus_MU_A.append(x_minus_mu_A)
                    X_minus_MU_B.append(x_minus_mu_B)

                X_minus_MU_A = np.array(X_minus_MU_A).reshape(-1,1)
                X_minus_MU_B = np.array(X_minus_MU_B).reshape(-1,1)

                f[r_l, ni, r_r] += IC_Cf_soft[indx[-1][ni], 0] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_B))) * np.exp( -0.5*(X_minus_MU_B).T @ np.linalg.inv(IC_cov_B) @ (X_minus_MU_B) ) \
                     + IC_Cf_soft[indx[-1][ni], 1] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_A))) * np.exp( -0.5*(X_minus_MU_A).T @ np.linalg.inv(IC_cov_A) @ (X_minus_MU_A) )

        if dim-4 < mode < dim-1:
            f = np.zeros((d1, d2, d3))
            for r_l, ni, r_r in it.product(range(d1), range(d2), range(d3)):
                X_minus_MU_A, X_minus_MU_B = [], []
                for ii in range(0,dim-3):
                    x_minus_mu_A =  Phi[indx[ii][r_l]] - IC_mu_A[ii]
                    x_minus_mu_B =  Phi[indx[ii][r_l]] - IC_mu_B[ii]
                    X_minus_MU_A.append(x_minus_mu_A)
                    X_minus_MU_B.append(x_minus_mu_B)

                X_minus_MU_A = np.array(X_minus_MU_A).reshape(-1,1)
                X_minus_MU_B = np.array(X_minus_MU_B).reshape(-1,1)

                f[r_l, ni, r_r] += IC_Cf_soft[indx[-1][r_r], 0] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_B))) * np.exp( -0.5*(X_minus_MU_B).T @ np.linalg.inv(IC_cov_B) @ (X_minus_MU_B) ) \
                     + IC_Cf_soft[indx[-1][r_r], 1] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_A))) * np.exp( -0.5*(X_minus_MU_A).T @ np.linalg.inv(IC_cov_A) @ (X_minus_MU_A) )
                
        if mode < dim-3:
            f = np.zeros((d1, d2, d3))
            for r_l, ni, r_r in it.product(range(d1), range(d2), range(d3)):
                X_minus_MU_A, X_minus_MU_B = [], []
                for ii in range(0,mode):
                    x_minus_mu_A =  Phi[indx[ii][r_l]] - IC_mu_A[ii]
                    x_minus_mu_B =  Phi[indx[ii][r_l]] - IC_mu_B[ii]
                    X_minus_MU_A.append(x_minus_mu_A)
                    X_minus_MU_B.append(x_minus_mu_B)

                X_minus_MU_A.append(Phi[indx[mode][ni]] - IC_mu_A[mode])
                X_minus_MU_B.append(Phi[indx[mode][ni]] - IC_mu_B[mode])

                for ii in range(mode+1,dim-3):
                    x_minus_mu_A =  Phi[indx[ii][r_r]] - IC_mu_A[ii]
                    x_minus_mu_B =  Phi[indx[ii][r_r]] - IC_mu_B[ii]
                    X_minus_MU_A.append(x_minus_mu_A)
                    X_minus_MU_B.append(x_minus_mu_B)

                X_minus_MU_A = np.array(X_minus_MU_A).reshape(-1,1)
                X_minus_MU_B = np.array(X_minus_MU_B).reshape(-1,1)

                f[r_l, ni, r_r] += IC_Cf_soft[indx[-1][r_r], 0] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_B))) * np.exp( -0.5*(X_minus_MU_B).T @ np.linalg.inv(IC_cov_B) @ (X_minus_MU_B) ) \
                     + IC_Cf_soft[indx[-1][r_r], 1] * (1 / np.sqrt((2*np.pi)**(dim-3)*np.linalg.det(IC_cov_A))) * np.exp( -0.5*(X_minus_MU_A).T @ np.linalg.inv(IC_cov_A) @ (X_minus_MU_A) )

    return f



# TT-CUR-DEIM Algorithm

def TT_DEIM(r, dim, max_iter):
  global N, All_D2, IC_mu_A, IC_mu_B, IC_cov_A, IC_cov_B, IC_mu_soft
  J = []
  [J.append(np.random.randint(0, N[j], size=r[i])) for i in range(dim-1) for j in range(i+1, dim)]  # Random right-nested indices for the first iteration

  for k in range(0,max_iter):   # Stopping criteria can be based on a convergence
      print('iteration:',k)
      Cores_current = []
      Cores_next = []
      I = []
      temp_idx = []
      temp_idx.append(np.arange(N[0]));    temp_idx.extend(J[0:dim-1]);
      C_first = np.squeeze(eval_input(temp_idx, k, 0, 1, N[0], len(temp_idx[1])))
      U_first,_,_ = np.linalg.svd(C_first, full_matrices=False)

      I.append(myDEIM(U_first, r[0]))
      G = (U_first[:, 0:r[0]] @ np.linalg.pinv(U_first[I[0],  0:r[0]])).reshape(1,N[0],r[0]) # Determines the first Core tensor of size 1*N*r
      Cores_current.append(G)   
      Cores_next.append(np.moveaxis(G, [0,2], [2,0]))   # Transpose the core for the next time step

      for z in range(1, dim-1):
          temp_idx = []
          temp_idx.extend(I[-z:]);   temp_idx.append(np.arange(N[z]));    temp_idx.extend(J[int(z*dim-0.5*(z**2+z)) : int((z+1)*dim-0.5*(z**2+3*z+2))])
          C_mid = eval_input(temp_idx, k, z, r[z-1], N[z], len(temp_idx[z+1]))
          C_mid = C_mid.reshape(r[z-1]*N[z], -1)
          U_mid,_,_ = np.linalg.svd(C_mid, full_matrices=False)
          I_combined = myDEIM(U_mid, r[z])
          [[I.append(np.array(I[int((z*(z-1))/2+ln)][np.array(np.floor(I_combined/N[z]), int)], int))]  for ln in range(z)]
          I.append(np.array(I_combined - N[z]*np.floor(I_combined/N[z]), int))
          G = (U_mid[:, 0:r[z]] @ np.linalg.pinv(U_mid[I_combined, 0:r[z]])).reshape(r[z-1],N[z],r[z]) # Determines the middle Core tensors of size rN*r
          Cores_current.append(G)
          Cores_next.append(np.moveaxis(G, [0,2], [2,0]))   # Flip the axis for the nest time step

      temp_idx = []
      temp_idx.extend(I[-(dim-1):]);   temp_idx.append(np.arange(N[dim-1]));
      C_end = (eval_input(temp_idx, k, dim-1, r[-1], N[dim-1], 1))
      Cores_current.append(C_end)   # Determines the last Core tensor of size r*N*1
      Cores_next.append(np.moveaxis(C_end, [0,2], [2,0]))

      J = I[::-1]  # Flip left-nested indices and use them as right-nested indices in the reverse iteration
      N = N[::-1]   # Flip N to be used in the reverse iteration based on the correct order
      IC_mu_A = IC_mu_A[::-1];   IC_mu_B = IC_mu_B[::-1];
      IC_cov_A = np.flip(np.flip(IC_cov_A,0), 1);    IC_cov_B = np.flip(np.flip(IC_cov_B,0), 1)
      r = r[::-1]
      All_D2 = All_D2[::-1]

  return Cores_current, Cores_next[::-1],  J



# ============================================================================================================
# A function to evaluate the Right-Hand side of the PDE in DEIM-selected fibers based on the sweep direction and the mode

def Eval_RHS(iter_count, mode, C_RK, G0_F, G0, indx, d1, d2, d3):      

    if np.remainder(iter_count,2) == 1:   # Performing Forward Iteration, it is oppsite of the eval_input function

        if mode == 0:
            
            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm = G0[mode].shape[2]
            rm_F = G0_F[mode].shape[2]
            for jj in range(3):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj] 
                New_G0_F_term1[jj] = D1x @ G0_F[jj] 
                New_G0_term21[jj] = D1x @ G0[jj] 
                New_G0_F_term21[jj] = D1x @ G0_F[jj] 
                New_G0_term22[jj] = D2x @ G0[jj] 
                New_G0_F_term22[jj] = D2x @ G0_F[jj] 

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(-1, rm)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(-1, rm_F)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(-1, rm)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(-1, rm_F)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(-1, rm)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(-1, rm_F)
            
                Right_terms_term1 = np.moveaxis(New_G0_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term1 = np.moveaxis(New_G0_F_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term21 = np.moveaxis(New_G0_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term21 = np.moveaxis(New_G0_F_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term22 = np.moveaxis(New_G0_term22[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term22 = np.moveaxis(New_G0_F_term22[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2,dim):
                    Right_terms_term1 = Right_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term1 = Right_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term21 = Right_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term21 = Right_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term22 = Right_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term22 = Right_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)


                Term1 += vel[jj][indx[0].reshape(-1,1), indx[1].reshape(1,-1), indx[2].reshape(1,-1)] * ( Mid_term_term1 @ np.squeeze(Right_terms_term1, axis=2).T + C_RK*dt* (Mid_term_F_term1 @ np.squeeze(Right_terms_F_term1, axis=2).T) ).reshape(d1,d2,d3)
                Term2 += Nu_derivation[jj][indx[0].reshape(-1,1), indx[1].reshape(1,-1), indx[2].reshape(1,-1)] * ( Mid_term_term21 @ np.squeeze(Right_terms_term21, axis=2).T + C_RK*dt* (Mid_term_F_term21 @ np.squeeze(Right_terms_F_term21, axis=2).T) ).reshape(d1,d2,d3) \
                      + Nu[indx[0].reshape(-1,1), indx[1].reshape(1,-1), indx[2].reshape(1,-1)] * ( Mid_term_term22 @ np.squeeze(Right_terms_term22, axis=2).T + C_RK*dt* (Mid_term_F_term22 @ np.squeeze(Right_terms_F_term22, axis=2).T) ).reshape(d1,d2,d3)

            
            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            First_3G = (G0[0][:,indx[0],:] @ (G0[1][:,indx[1],:] @ G0[2][:,indx[2],:].reshape(G0[2].shape[0], -1)).reshape(G0[1].shape[0], -1)).reshape(-1, G0[2].shape[2])
            First_3G_F = (G0_F[0][:,indx[0],:] @ (G0_F[1][:,indx[1],:] @ G0_F[2][:,indx[2],:].reshape(G0_F[2].shape[0], -1)).reshape(G0_F[1].shape[0], -1)).reshape(-1, G0_F[2].shape[2])
            for jj in range(3,dim):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = First_3G[:]+0;   Avg_Phi_F = First_3G_F[:]+0
                for ii in range(3, dim):
                    Avg_Phi = Avg_Phi @ np.sum(Delta_Phi*New_G0_avg[ii], axis=1)
                    Avg_Phi_F = Avg_Phi_F @ np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1)

                Avg_Phi = Avg_Phi.reshape(N[0], len(indx[1]), len(indx[2]))
                Avg_Phi_F = Avg_Phi_F.reshape(N[0], len(indx[1]), len(indx[2]))

                Avg_idx = indx.copy()
                Avg_idx[1] = np.arange(len(indx[1]));   Avg_idx[2] = np.arange(len(indx[2]))

    
                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])       # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]               # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj] 
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0[jj])
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0_F[jj])

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(-1, rm)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(-1, rm_F)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(-1, rm)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(-1, rm_F)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(-1, rm)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(-1, rm_F)

                Right_terms_term31 = np.moveaxis(New_G0_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term31 = np.moveaxis(New_G0_F_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term32 = np.moveaxis(New_G0_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term32 = np.moveaxis(New_G0_F_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term4 = np.moveaxis(New_G0_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term4 = np.moveaxis(New_G0_F_term4[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2, dim):
                    Right_terms_term31 = Right_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term31 = Right_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term32 = Right_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term32 = Right_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term4 = Right_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term4 = Right_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)
                
                Term3 += Omega[indx[0].reshape(-1,1), indx[1].reshape(1,-1), indx[2].reshape(1,-1)] \
                        * ( (Mid_term_term31 @ np.squeeze(Right_terms_term31, axis=2).T).reshape(d1,d2,d3) \
                        -  Avg_Phi[Avg_idx[0].reshape(-1,1), Avg_idx[1].reshape(1,-1), Avg_idx[2].reshape(1,-1)] * (Mid_term_term32 @ np.squeeze(Right_terms_term32, axis=2).T).reshape(d1,d2,d3) \
                            + C_RK*dt* ( (Mid_term_F_term31 @ np.squeeze(Right_terms_F_term31, axis=2).T).reshape(d1,d2,d3) \
                                        - Avg_Phi_F[Avg_idx[0].reshape(-1,1), Avg_idx[1].reshape(1,-1), Avg_idx[2].reshape(1,-1)] * (Mid_term_F_term32 @ np.squeeze(Right_terms_F_term32, axis=2).T).reshape(d1,d2,d3) ) )
                    
                Term4 += ( Mid_term_term4 @ np.squeeze(Right_terms_term4, axis=2).T + C_RK*dt* (Mid_term_F_term4 @ np.squeeze(Right_terms_F_term4, axis=2).T) ).reshape(d1,d2,d3)


            F = -1*Term1 + Term2 + Term3 - Term4

                    


        if mode == 1:
            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm1 = G0[mode].shape[0]
            rm2 = G0[mode].shape[2]
            rm_F1 = G0_F[mode].shape[0]
            rm_F2 = G0_F[mode].shape[2]
            for jj in range(3):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj] 
                New_G0_F_term1[jj] = D1x @ G0_F[jj]
                New_G0_term21[jj] = D1x @ G0[jj] 
                New_G0_F_term21[jj] = D1x @ G0_F[jj] 
                New_G0_term22[jj] = D2x @ G0[jj] 
                New_G0_F_term22[jj] = D2x @ G0_F[jj]

                Left_terms_term1 = New_G0_term1[0][:,New_idx[0],:]
                Left_terms_F_term1 = New_G0_F_term1[0][:,New_idx[0],:]
                Left_terms_term21 = New_G0_term21[0][:,New_idx[0],:]
                Left_terms_F_term21 = New_G0_F_term21[0][:,New_idx[0],:]
                Left_terms_term22 = New_G0_term22[0][:,New_idx[0],:]
                Left_terms_F_term22 = New_G0_F_term22[0][:,New_idx[0],:]

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(rm1, -1)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(rm_F1, -1)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(rm1, -1)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(rm_F1, -1)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(rm1, -1)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(rm_F1, -1)

                Right_terms_term1 = np.moveaxis(New_G0_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term1 = np.moveaxis(New_G0_F_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term21 = np.moveaxis(New_G0_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term21 = np.moveaxis(New_G0_F_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term22 = np.moveaxis(New_G0_term22[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term22 = np.moveaxis(New_G0_F_term22[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2,dim):
                    Right_terms_term1 = Right_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term1 = Right_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term21 = Right_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term21 = Right_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term22 = Right_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term22 = Right_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)
                
 
                Term1 += vel[jj][indx[0].reshape(-1,1,1), indx[1].reshape(1,-1,1), indx[2].reshape(1,1,-1)] \
                       * ( (np.squeeze(Left_terms_term1, axis=0) @ Mid_term_term1).reshape(-1,rm2) @ np.squeeze(Right_terms_term1, axis=2).T \
                          +  C_RK*dt* ( (np.squeeze(Left_terms_F_term1, axis=0) @ Mid_term_F_term1).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term1, axis=2).T ) ).reshape(d1,d2,d3)
                
                Term2 += Nu_derivation[jj][indx[0].reshape(-1,1,1), indx[1].reshape(1,-1,1), indx[2].reshape(1,1,-1)] \
                       * ( (np.squeeze(Left_terms_term21, axis=0) @ Mid_term_term21).reshape(-1,rm2) @ np.squeeze(Right_terms_term21, axis=2).T \
                          +  C_RK*dt* ( (np.squeeze(Left_terms_F_term21, axis=0) @ Mid_term_F_term21).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term21, axis=2).T ) ).reshape(d1,d2,d3) \
                        + Nu[indx[0].reshape(-1,1,1), indx[1].reshape(1,-1,1), indx[2].reshape(1,1,-1)] \
                       * (  (np.squeeze(Left_terms_term22, axis=0) @ Mid_term_term22).reshape(-1,rm2) @ np.squeeze(Right_terms_term22, axis=2).T \
                          +  C_RK*dt* ( (np.squeeze(Left_terms_F_term22, axis=0) @ Mid_term_F_term22).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term22, axis=2).T )  ).reshape(d1,d2,d3)
                 
            
            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            First_3G = (G0[0][:,indx[0],:] @ (G0[1][:,indx[1],:] @ G0[2][:,indx[2],:].reshape(G0[2].shape[0], -1)).reshape(G0[1].shape[0], -1)).reshape(-1, G0[2].shape[2])
            First_3G_F = (G0_F[0][:,indx[0],:] @ (G0_F[1][:,indx[1],:] @ G0_F[2][:,indx[2],:].reshape(G0_F[2].shape[0], -1)).reshape(G0_F[1].shape[0], -1)).reshape(-1, G0_F[2].shape[2])
            for jj in range(3,dim):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = First_3G[:]+0;   Avg_Phi_F = First_3G_F[:]+0
                for ii in range(3, dim):
                    Avg_Phi = Avg_Phi @ np.sum(Delta_Phi*New_G0_avg[ii], axis=1)
                    Avg_Phi_F = Avg_Phi_F @ np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1)
      
                Avg_Phi = Avg_Phi.reshape(len(indx[0]), N[1], len(indx[2]))
                Avg_Phi_F = Avg_Phi_F.reshape(len(indx[0]), N[1], len(indx[2]))
                Avg_idx = indx.copy()
                Avg_idx[0] = np.arange(len(indx[0]));   Avg_idx[2] = np.arange(len(indx[2]))

                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])      # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]     # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj] 
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0[jj]) 
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0_F[jj])

                Left_terms_term31 = New_G0_term31[0][:,New_idx[0],:]
                Left_terms_F_term31 = New_G0_F_term31[0][:,New_idx[0],:]
                Left_terms_term32 = New_G0_term32[0][:,New_idx[0],:]
                Left_terms_F_term32 = New_G0_F_term32[0][:,New_idx[0],:]
                Left_terms_term4 = New_G0_term4[0][:,New_idx[0],:]
                Left_terms_F_term4 = New_G0_F_term4[0][:,New_idx[0],:]

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(rm1, -1)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(rm_F1, -1)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(rm1, -1)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(rm_F1, -1)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(rm1, -1)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(rm_F1, -1)

                Right_terms_term31 = np.moveaxis(New_G0_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term31 = np.moveaxis(New_G0_F_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term32 = np.moveaxis(New_G0_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term32 = np.moveaxis(New_G0_F_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term4 = np.moveaxis(New_G0_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term4 = np.moveaxis(New_G0_F_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                    
                for ii in range(mode+2, dim):
                    Right_terms_term31 = Right_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term31 = Right_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term32 = Right_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term32 = Right_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term4 = Right_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term4 = Right_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)
                

                Term3 += Omega[indx[0].reshape(-1,1,1), indx[1].reshape(1,-1,1), indx[2].reshape(1,1,-1)] \
                        * ( ((np.squeeze(Left_terms_term31, axis=0) @ Mid_term_term31).reshape(-1,rm2) @ np.squeeze(Right_terms_term31, axis=2).T).reshape(d1,d2,d3) \
                           - Avg_Phi[Avg_idx[0].reshape(-1,1,1), Avg_idx[1].reshape(1,-1,1), Avg_idx[2].reshape(1,1,-1)] * ( (np.squeeze(Left_terms_term32, axis=0) @ Mid_term_term32).reshape(-1, rm2) @ np.squeeze(Right_terms_term32, axis=2).T ).reshape(d1,d2,d3) \
                        + C_RK*dt* ( ((np.squeeze(Left_terms_F_term31, axis=0) @ Mid_term_F_term31).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term31, axis=2).T).reshape(d1,d2,d3) \
                                      - Avg_Phi_F[Avg_idx[0].reshape(-1,1,1), Avg_idx[1].reshape(1,-1,1), Avg_idx[2].reshape(1,1,-1)] * ( (np.squeeze(Left_terms_F_term32, axis=0) @ Mid_term_F_term32).reshape(-1, rm_F2) @ np.squeeze(Right_terms_F_term32, axis=2).T ).reshape(d1,d2,d3)  )  )
            
                Term4 += ( (np.squeeze(Left_terms_term4, axis=0) @ Mid_term_term4).reshape(-1,rm2) @ np.squeeze(Right_terms_term4, axis=2).T \
                         + C_RK*dt* ( (np.squeeze(Left_terms_F_term4, axis=0) @ Mid_term_F_term4).reshape(-1, rm_F2) @ np.squeeze(Right_terms_F_term4, axis=2).T )   ).reshape(d1,d2,d3)
            

            F = -1*Term1 + Term2 + Term3 - Term4
        
        

        if mode == 2:

            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm1 = G0[mode].shape[0]
            rm2 = G0[mode].shape[2]
            rm_F1 = G0_F[mode].shape[0]
            rm_F2 = G0_F[mode].shape[2]
            for jj in range(3):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj] 
                New_G0_F_term1[jj] = D1x @ G0_F[jj] 
                New_G0_term21[jj] = D1x @ G0[jj] 
                New_G0_F_term21[jj] = D1x @ G0_F[jj]
                New_G0_term22[jj] = D2x @ G0[jj]
                New_G0_F_term22[jj] = D2x @ G0_F[jj]

                Left_terms_term1 = np.moveaxis(New_G0_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term1 = np.moveaxis(New_G0_F_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_term21 = np.moveaxis(New_G0_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term21 = np.moveaxis(New_G0_F_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_term22 = np.moveaxis(New_G0_term22[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term22 = np.moveaxis(New_G0_F_term22[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term1 = Left_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term1 = Left_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term21 = Left_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term21 = Left_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term22 = Left_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term22 = Left_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(rm1, -1)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(rm_F1, -1)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(rm1, -1)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(rm_F1, -1)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(rm1, -1)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(rm_F1, -1)

                Right_terms_term1 = np.moveaxis(New_G0_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term1 = np.moveaxis(New_G0_F_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term21 = np.moveaxis(New_G0_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term21 = np.moveaxis(New_G0_F_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term22 = np.moveaxis(New_G0_term22[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term22 = np.moveaxis(New_G0_F_term22[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2,dim):
                    Right_terms_term1 = Right_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term1 = Right_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term21 = Right_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term21 = Right_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term22 = Right_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term22 = Right_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)


                Term1 += ( vel[jj][indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(1,-1)].reshape(-1,1) \
                       * ( (np.squeeze(Left_terms_term1, axis=1) @ Mid_term_term1).reshape(-1,rm2) @ np.squeeze(Right_terms_term1, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term1, axis=1) @ Mid_term_F_term1).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term1, axis=2).T ) )  ).reshape(d1,d2,d3)
                
                Term2 += ( Nu_derivation[jj][indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(1,-1)].reshape(-1,1) \
                       * ( (np.squeeze(Left_terms_term21, axis=1) @ Mid_term_term21).reshape(-1,rm2) @ np.squeeze(Right_terms_term21, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term21, axis=1) @ Mid_term_F_term21).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term21, axis=2).T  ) ) \
                        + Nu[indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(1,-1)].reshape(-1,1) \
                        * ( (np.squeeze(Left_terms_term22, axis=1) @ Mid_term_term22).reshape(-1,rm2) @ np.squeeze(Right_terms_term22, axis=2).T \
                           + C_RK*dt* ( (np.squeeze(Left_terms_F_term22, axis=1) @ Mid_term_F_term22).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term22, axis=2).T  ) )   ).reshape(d1,d2,d3)
    
            
            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            First_3G = (G0[0][:,indx[0],:] @ (G0[1][:,indx[1],:] @ G0[2][:,indx[2],:].reshape(G0[2].shape[0], -1)).reshape(G0[1].shape[0], -1)).reshape(-1, G0[2].shape[2])
            First_3G_F = (G0_F[0][:,indx[0],:] @ (G0_F[1][:,indx[1],:] @ G0_F[2][:,indx[2],:].reshape(G0_F[2].shape[0], -1)).reshape(G0_F[1].shape[0], -1)).reshape(-1, G0_F[2].shape[2])
            for jj in range(3,dim):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = First_3G[:]+0;   Avg_Phi_F = First_3G_F[:]+0
                for ii in range(3, dim):
                    Avg_Phi = Avg_Phi @ np.sum(Delta_Phi*New_G0_avg[ii], axis=1)
                    Avg_Phi_F = Avg_Phi_F @ np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1)
      
                Avg_Phi = Avg_Phi.reshape(len(indx[0]), len(indx[1]), N[2])
                Avg_Phi_F = Avg_Phi_F.reshape(len(indx[0]), len(indx[1]), N[2])
                Avg_idx = indx.copy()
                Avg_idx[0] = np.arange(len(indx[0]));   Avg_idx[1] = np.arange(len(indx[1]))

                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])      # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]       # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj] 
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0[jj]) 
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0_F[jj])

                Left_terms_term31 = np.moveaxis(New_G0_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term31 = np.moveaxis(New_G0_F_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_term32 = np.moveaxis(New_G0_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term32 = np.moveaxis(New_G0_F_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_term4 = np.moveaxis(New_G0_term4[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term4 = np.moveaxis(New_G0_F_term4[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term31 = Left_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term31 = Left_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term32 = Left_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term32 = Left_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term4 = Left_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term4 = Left_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(rm1, -1)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(rm_F1, -1)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(rm1, -1)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(rm_F1, -1)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(rm1, -1)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(rm_F1, -1)

                Right_terms_term31 = np.moveaxis(New_G0_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term31 = np.moveaxis(New_G0_F_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term32 = np.moveaxis(New_G0_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term32 = np.moveaxis(New_G0_F_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term4 = np.moveaxis(New_G0_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term4 = np.moveaxis(New_G0_F_term4[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2, dim):
                    Right_terms_term31 = Right_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term31 = Right_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term32 = Right_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term32 = Right_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term4 = Right_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term4 = Right_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)
                    
                Term3 += ( Omega[indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(1,-1)].reshape(-1,1) \
                       * ( (np.squeeze(Left_terms_term31, axis=1) @ Mid_term_term31).reshape(-1,rm2) @ np.squeeze(Right_terms_term31, axis=2).T \
                          - Avg_Phi[Avg_idx[0].reshape(-1,1), Avg_idx[1].reshape(-1,1), Avg_idx[2].reshape(1,-1)].reshape(-1,1) * ( (np.squeeze(Left_terms_term32, axis=1) @ Mid_term_term32).reshape(-1,rm2) @ np.squeeze(Right_terms_term32, axis=2).T ) \
                        + C_RK*dt* ( (np.squeeze(Left_terms_F_term31, axis=1) @ Mid_term_F_term31).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term31, axis=2).T \
                                    - Avg_Phi_F[Avg_idx[0].reshape(-1,1), Avg_idx[1].reshape(-1,1), Avg_idx[2].reshape(1,-1)].reshape(-1,1) * ( (np.squeeze(Left_terms_F_term32, axis=1) @ Mid_term_F_term32).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term32, axis=2).T )  )  )   ).reshape(d1,d2,d3)
                                 
                Term4 += ( (np.squeeze(Left_terms_term4, axis=1) @ Mid_term_term4).reshape(-1,rm2) @ np.squeeze(Right_terms_term4, axis=2).T \
                         + C_RK*dt* ( (np.squeeze(Left_terms_F_term4, axis=1) @ Mid_term_F_term4).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term4, axis=2).T )   ).reshape(d1,d2,d3)
                
            F = -1*Term1 + Term2 + Term3 - Term4



        if 2.5 < mode < dim-1.5:

            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm1 = G0[mode].shape[0]
            rm2 = G0[mode].shape[2]
            rm_F1 = G0_F[mode].shape[0]
            rm_F2 = G0_F[mode].shape[2]
            for jj in range(3):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj] 
                New_G0_F_term1[jj] = D1x @ G0_F[jj]
                New_G0_term21[jj] = D1x @ G0[jj] 
                New_G0_F_term21[jj] = D1x @ G0_F[jj] 
                New_G0_term22[jj] = D2x @ G0[jj] 
                New_G0_F_term22[jj] = D2x @ G0_F[jj] 
                
                Left_terms_term1 = np.moveaxis(New_G0_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term1 = np.moveaxis(New_G0_F_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_term21 = np.moveaxis(New_G0_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term21 = np.moveaxis(New_G0_F_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_term22 = np.moveaxis(New_G0_term22[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term22 = np.moveaxis(New_G0_F_term22[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term1 = Left_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term1 = Left_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term21 = Left_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term21 = Left_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term22 = Left_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term22 = Left_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(rm1, -1)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(rm_F1, -1)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(rm1, -1)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(rm_F1, -1)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(rm1, -1)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(rm_F1, -1)

                Right_terms_term1 = np.moveaxis(New_G0_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term1 = np.moveaxis(New_G0_F_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term21 = np.moveaxis(New_G0_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term21 = np.moveaxis(New_G0_F_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term22 = np.moveaxis(New_G0_term22[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term22 = np.moveaxis(New_G0_F_term22[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2,dim):
                    Right_terms_term1 = Right_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term1 = Right_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term21 = Right_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term21 = Right_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term22 = Right_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term22 = Right_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)


                Term1 += ( vel[jj][indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(-1,1)] \
                       * ( (np.squeeze(Left_terms_term1, axis=1) @ Mid_term_term1).reshape(-1,rm2) @ np.squeeze(Right_terms_term1, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term1, axis=1) @ Mid_term_F_term1).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term1, axis=2).T ) ).reshape(d1,-1)  ).reshape(d1,d2,d3)
                
                Term2 += ( Nu_derivation[jj][indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(-1,1)] \
                       * ( (np.squeeze(Left_terms_term21, axis=1) @ Mid_term_term21).reshape(-1,rm2) @ np.squeeze(Right_terms_term21, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term21, axis=1) @ Mid_term_F_term21).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term21, axis=2).T  ) ).reshape(d1,-1) \
                        + Nu[indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(-1,1)] \
                        * ( (np.squeeze(Left_terms_term22, axis=1) @ Mid_term_term22).reshape(-1,rm2) @ np.squeeze(Right_terms_term22, axis=2).T \
                           + C_RK*dt* ( (np.squeeze(Left_terms_F_term22, axis=1) @ Mid_term_F_term22).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term22, axis=2).T  ) ).reshape(d1,-1)  ).reshape(d1,d2,d3)
    
            
            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            First_3G = (G0[0][:,indx[0],:] @ (G0[1][:,indx[1],:] @ G0[2][:,indx[2],:].reshape(G0[2].shape[0], -1)).reshape(G0[1].shape[0], -1)).reshape(-1, G0[2].shape[2])
            First_3G_F = (G0_F[0][:,indx[0],:] @ (G0_F[1][:,indx[1],:] @ G0_F[2][:,indx[2],:].reshape(G0_F[2].shape[0], -1)).reshape(G0_F[1].shape[0], -1)).reshape(-1, G0_F[2].shape[2])
            for jj in range(3,dim):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = First_3G[:]+0;   Avg_Phi_F = First_3G_F[:]+0
                for ii in range(3, dim):
                    Avg_Phi = Avg_Phi @ np.sum(Delta_Phi*New_G0_avg[ii], axis=1)
                    Avg_Phi_F = Avg_Phi_F @ np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1)
      
                Avg_Phi = Avg_Phi.reshape(len(indx[0]), len(indx[1]), len(indx[2]))
                Avg_Phi_F = Avg_Phi_F.reshape(len(indx[0]), len(indx[1]), len(indx[2]))
                Avg_idx = indx.copy()
                Avg_idx[0] = np.arange(len(indx[0]));   Avg_idx[1] = np.arange(len(indx[1]));   Avg_idx[2] = np.arange(len(indx[2]))

                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])       # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]         # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj] 
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0[jj]) 
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0_F[jj]) 

                Left_terms_term31 = np.moveaxis(New_G0_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term31 = np.moveaxis(New_G0_F_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_term32 = np.moveaxis(New_G0_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term32 = np.moveaxis(New_G0_F_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_term4 = np.moveaxis(New_G0_term4[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term4 = np.moveaxis(New_G0_F_term4[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term31 = Left_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term31 = Left_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term32 = Left_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term32 = Left_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term4 = Left_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term4 = Left_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(rm1, -1)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(rm_F1, -1)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(rm1, -1)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(rm_F1, -1)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(rm1, -1)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(rm_F1, -1)

                Right_terms_term31 = np.moveaxis(New_G0_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term31 = np.moveaxis(New_G0_F_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term32 = np.moveaxis(New_G0_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term32 = np.moveaxis(New_G0_F_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term4 = np.moveaxis(New_G0_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term4 = np.moveaxis(New_G0_F_term4[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2, dim):
                    Right_terms_term31 = Right_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term31 = Right_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term32 = Right_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term32 = Right_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term4 = Right_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term4 = Right_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)
                    
                Term3 += ( Omega[indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(-1,1)] \
                       * ( ((np.squeeze(Left_terms_term31, axis=1) @ Mid_term_term31).reshape(-1,rm2) @ np.squeeze(Right_terms_term31, axis=2).T).reshape(d1,-1) \
                          - Avg_Phi[Avg_idx[0].reshape(-1,1), Avg_idx[1].reshape(-1,1), Avg_idx[2].reshape(-1,1)] * ( (np.squeeze(Left_terms_term32, axis=1) @ Mid_term_term32).reshape(-1,rm2) @ np.squeeze(Right_terms_term32, axis=2).T ).reshape(d1,-1) \
                        + C_RK*dt* ( ((np.squeeze(Left_terms_F_term31, axis=1) @ Mid_term_F_term31).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term31, axis=2).T).reshape(d1,-1) \
                                    - Avg_Phi_F[Avg_idx[0].reshape(-1,1), Avg_idx[1].reshape(-1,1), Avg_idx[2].reshape(-1,1)] * ( (np.squeeze(Left_terms_F_term32, axis=1) @ Mid_term_F_term32).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term32, axis=2).T ).reshape(d1,-1)  )  )   ).reshape(d1,d2,d3)
                                 
                Term4 += ( (np.squeeze(Left_terms_term4, axis=1) @ Mid_term_term4).reshape(-1,rm2) @ np.squeeze(Right_terms_term4, axis=2).T \
                         + C_RK*dt* ( (np.squeeze(Left_terms_F_term4, axis=1) @ Mid_term_F_term4).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term4, axis=2).T )   ).reshape(d1,d2,d3)
                    
                      
            F = -1*Term1 + Term2 + Term3 - Term4



        if  mode == dim-1:

            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm1 = G0[mode].shape[0]
            rm2 = G0[mode].shape[2]
            rm_F1 = G0_F[mode].shape[0]
            rm_F2 = G0_F[mode].shape[2]
            for jj in range(3):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj] 
                New_G0_F_term1[jj] = D1x @ G0_F[jj]
                New_G0_term21[jj] = D1x @ G0[jj] 
                New_G0_F_term21[jj] = D1x @ G0_F[jj] 
                New_G0_term22[jj] = D2x @ G0[jj] 
                New_G0_F_term22[jj] = D2x @ G0_F[jj] 
                
                Left_terms_term1 = np.moveaxis(New_G0_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term1 = np.moveaxis(New_G0_F_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_term21 = np.moveaxis(New_G0_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term21 = np.moveaxis(New_G0_F_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_term22 = np.moveaxis(New_G0_term22[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term22 = np.moveaxis(New_G0_F_term22[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term1 = Left_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term1 = Left_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term21 = Left_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term21 = Left_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term22 = Left_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term22 = Left_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(rm1, -1)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(rm_F1, -1)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(rm1, -1)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(rm_F1, -1)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(rm1, -1)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(rm_F1, -1)


                Term1 += ( vel[jj][indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(-1,1)] \
                       * ( np.squeeze(Left_terms_term1, axis=1) @ Mid_term_term1 \
                          + C_RK*dt* ( np.squeeze(Left_terms_F_term1, axis=1) @ Mid_term_F_term1 ) )  ).reshape(d1,d2,d3)
                
                Term2 += ( Nu_derivation[jj][indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(-1,1)] \
                       * ( np.squeeze(Left_terms_term21, axis=1) @ Mid_term_term21 \
                          + C_RK*dt* ( np.squeeze(Left_terms_F_term21, axis=1) @ Mid_term_F_term21 ) ) \
                        + Nu[indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(-1,1)] \
                        * ( np.squeeze(Left_terms_term22, axis=1) @ Mid_term_term22 \
                           + C_RK*dt* ( np.squeeze(Left_terms_F_term22, axis=1) @ Mid_term_F_term22 ) )  ).reshape(d1,d2,d3)
    
            
            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            First_3G = (G0[0][:,indx[0],:] @ (G0[1][:,indx[1],:] @ G0[2][:,indx[2],:].reshape(G0[2].shape[0], -1)).reshape(G0[1].shape[0], -1)).reshape(-1, G0[2].shape[2])
            First_3G_F = (G0_F[0][:,indx[0],:] @ (G0_F[1][:,indx[1],:] @ G0_F[2][:,indx[2],:].reshape(G0_F[2].shape[0], -1)).reshape(G0_F[1].shape[0], -1)).reshape(-1, G0_F[2].shape[2])
            for jj in range(3,dim):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = First_3G[:]+0;   Avg_Phi_F = First_3G_F[:]+0
                for ii in range(3, dim):
                    Avg_Phi = Avg_Phi @ np.sum(Delta_Phi*New_G0_avg[ii], axis=1)
                    Avg_Phi_F = Avg_Phi_F @ np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1)
      
                Avg_Phi = Avg_Phi.reshape(len(indx[0]), len(indx[1]), len(indx[2]))
                Avg_Phi_F = Avg_Phi_F.reshape(len(indx[0]), len(indx[1]), len(indx[2]))
                Avg_idx = indx.copy()
                Avg_idx[0] = np.arange(len(indx[0]));   Avg_idx[1] = np.arange(len(indx[1]));   Avg_idx[2] = np.arange(len(indx[2]))

                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])       # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]         # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj] 
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0[jj]) 
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,jj-3].reshape(-1,1)*G0_F[jj]) 

                Left_terms_term31 = np.moveaxis(New_G0_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term31 = np.moveaxis(New_G0_F_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_term32 = np.moveaxis(New_G0_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term32 = np.moveaxis(New_G0_F_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_term4 = np.moveaxis(New_G0_term4[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term4 = np.moveaxis(New_G0_F_term4[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term31 = Left_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term31 = Left_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term32 = Left_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term32 = Left_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term4 = Left_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term4 = Left_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(rm1, -1)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(rm_F1, -1)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(rm1, -1)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(rm_F1, -1)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(rm1, -1)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(rm_F1, -1)


                Term3 += ( Omega[indx[0].reshape(-1,1), indx[1].reshape(-1,1), indx[2].reshape(-1,1)] \
                       * ( np.squeeze(Left_terms_term31, axis=1) @ Mid_term_term31 \
                          - Avg_Phi[Avg_idx[0].reshape(-1,1), Avg_idx[1].reshape(-1,1), Avg_idx[2].reshape(-1,1)] * ( np.squeeze(Left_terms_term32, axis=1) @ Mid_term_term32 ) \
                          + C_RK*dt* ( np.squeeze(Left_terms_F_term31, axis=1) @ Mid_term_F_term31 \
                                    - Avg_Phi_F[Avg_idx[0].reshape(-1,1), Avg_idx[1].reshape(-1,1), Avg_idx[2].reshape(-1,1)] * ( np.squeeze(Left_terms_F_term32, axis=1) @ Mid_term_F_term32)  )  )   ).reshape(d1,d2,d3)
                                 
                Term4 += ( np.squeeze(Left_terms_term4, axis=1) @ Mid_term_term4 \
                         + C_RK*dt* ( np.squeeze(Left_terms_F_term4, axis=1) @ Mid_term_F_term4 ) ).reshape(d1,d2,d3)
                    
                      
            F = -1*Term1 + Term2 + Term3 - Term4
                


    if np.remainder(iter_count,2) == 0:   # Performing Reverse Iteration, it is oppsite of the eval_input function

        if mode == 0:

            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm = G0[mode].shape[2]
            rm_F = G0_F[mode].shape[2]
            for jj in range(dim-3, dim):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj] 
                New_G0_F_term1[jj] = D1x @ G0_F[jj] 
                New_G0_term21[jj] = D1x @ G0[jj]
                New_G0_F_term21[jj] = D1x @ G0_F[jj] 
                New_G0_term22[jj] = D2x @ G0[jj]
                New_G0_F_term22[jj] = D2x @ G0_F[jj]

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(-1, rm)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(-1, rm_F)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(-1, rm)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(-1, rm_F)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(-1, rm)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(-1, rm_F)

                Right_terms_term1 = np.moveaxis(New_G0_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term1 = np.moveaxis(New_G0_F_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term21 = np.moveaxis(New_G0_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term21 = np.moveaxis(New_G0_F_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term22 = np.moveaxis(New_G0_term22[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term22 = np.moveaxis(New_G0_F_term22[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2,dim):
                    Right_terms_term1 = Right_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term1 = Right_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term21 = Right_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term21 = Right_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term22 = Right_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term22 = Right_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)


                Term1 += ( vel[dim-jj-1][indx[-1], indx[-2], indx[-3]].reshape(1,-1) * (Mid_term_term1 @ np.squeeze(Right_terms_term1, axis=2).T + C_RK*dt* (Mid_term_F_term1 @ np.squeeze(Right_terms_F_term1, axis=2).T) ) ).reshape(d1,d2,d3)
                Term2 += ( Nu_derivation[dim-jj-1][indx[-1], indx[-2], indx[-3]].reshape(1,-1) * ( Mid_term_term21 @ np.squeeze(Right_terms_term21, axis=2).T + C_RK*dt* (Mid_term_F_term21 @ np.squeeze(Right_terms_F_term21, axis=2).T) ) \
                        + Nu[indx[-1], indx[-2], indx[-3]].reshape(1,-1) * ( Mid_term_term22 @ np.squeeze(Right_terms_term22, axis=2).T + C_RK*dt* (Mid_term_F_term22 @ np.squeeze(Right_terms_F_term22, axis=2).T) )  ).reshape(d1,d2,d3) 
            

            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            Last_3G = (G0[-3][:,indx[-3],:] @ (G0[-2][:,indx[-2],:] @ G0[-1][:,indx[-1],:].reshape(G0[-1].shape[0], -1)).reshape(G0[-2].shape[0], -1)).reshape(G0[-3].shape[0],-1)
            Last_3G_F = (G0_F[-3][:,indx[-3],:] @ (G0_F[-2][:,indx[-2],:] @ G0_F[-1][:,indx[-1],:].reshape(G0_F[-1].shape[0], -1)).reshape(G0_F[-2].shape[0], -1)).reshape(G0_F[-3].shape[0],-1)
            for jj in range(0,dim-3):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = Last_3G[:]+0;   Avg_Phi_F = Last_3G_F[:]+0
                for ii in reversed(range(0,dim-3)):
                    Avg_Phi = np.sum(Delta_Phi*New_G0_avg[ii], axis=1) @ Avg_Phi 
                    Avg_Phi_F = np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1) @ Avg_Phi_F
      

                Avg_Phi = Avg_Phi.reshape(len(indx[-3]), len(indx[-2]), len(indx[-1]))
                Avg_Phi_F = Avg_Phi_F.reshape(len(indx[-3]), len(indx[-2]), len(indx[-1]))
                Avg_idx = indx.copy()
                Avg_idx[-3] = np.arange(len(indx[-3]));   Avg_idx[-2] = np.arange(len(indx[-2]));   Avg_idx[-1] = np.arange(len(indx[-1]))

                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])       # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]    # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj] 
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0[jj]) 
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0_F[jj])

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(-1, rm)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(-1, rm_F)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(-1, rm)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(-1, rm_F)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(-1, rm)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(-1, rm_F)

                Right_terms_term31 = np.moveaxis(New_G0_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term31 = np.moveaxis(New_G0_F_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term32 = np.moveaxis(New_G0_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term32 = np.moveaxis(New_G0_F_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term4 = np.moveaxis(New_G0_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term4 = np.moveaxis(New_G0_F_term4[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2, dim):
                    Right_terms_term31 = Right_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term31 = Right_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term32 = Right_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term32 = Right_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term4 = Right_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term4 = Right_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)
                
                Term3 += ( Omega[indx[-1], indx[-2], indx[-3]].reshape(1,-1) \
                           * ( (Mid_term_term31 @ np.squeeze(Right_terms_term31, axis=2).T) \
                                - Avg_Phi[Avg_idx[-3],Avg_idx[-2],Avg_idx[-1]].reshape(1,-1) * (Mid_term_term32 @ np.squeeze(Right_terms_term32, axis=2).T) \
                                    + C_RK*dt* ( (Mid_term_F_term31 @ np.squeeze(Right_terms_F_term31, axis=2).T) \
                                                - Avg_Phi[Avg_idx[-3],Avg_idx[-2],Avg_idx[-1]].reshape(1,-1) * (Mid_term_F_term32 @ np.squeeze(Right_terms_F_term32, axis=2).T)  ) )   ).reshape(d1,d2,d3)
                                 
                Term4 += ( Mid_term_term4 @ np.squeeze(Right_terms_term4, axis=2).T + C_RK*dt* (Mid_term_F_term4 @ np.squeeze(Right_terms_F_term4, axis=2).T) ).reshape(d1,d2,d3)
                        
            F = -1*Term1 + Term2 + Term3 - Term4
                    


        if 0.5 < mode < dim-3.5:

            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm1 = G0[mode].shape[0]
            rm2 = G0[mode].shape[2]
            rm_F1 = G0_F[mode].shape[0]
            rm_F2 = G0_F[mode].shape[2]
            for jj in range(dim-3, dim):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj]
                New_G0_F_term1[jj] = D1x @ G0_F[jj] 
                New_G0_term21[jj] = D1x @ G0[jj] 
                New_G0_F_term21[jj] = D1x @ G0_F[jj] 
                New_G0_term22[jj] = D2x @ G0[jj]
                New_G0_F_term22[jj] = D2x @ G0_F[jj] 

                Left_terms_term1 = np.moveaxis(New_G0_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term1 = np.moveaxis(New_G0_F_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_term21 = np.moveaxis(New_G0_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term21 = np.moveaxis(New_G0_F_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_term22 = np.moveaxis(New_G0_term22[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term22 = np.moveaxis(New_G0_F_term22[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term1 = Left_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term1 = Left_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term21 = Left_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term21 = Left_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term22 = Left_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term22 = Left_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(rm1, -1)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(rm_F1, -1)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(rm1, -1)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(rm_F1, -1)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(rm1, -1)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(rm_F1, -1)

                Right_terms_term1 = np.moveaxis(New_G0_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term1 = np.moveaxis(New_G0_F_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term21 = np.moveaxis(New_G0_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term21 = np.moveaxis(New_G0_F_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term22 = np.moveaxis(New_G0_term22[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term22 = np.moveaxis(New_G0_F_term22[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2,dim):
                    Right_terms_term1 = Right_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term1 = Right_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term21 = Right_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term21 = Right_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term22 = Right_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term22 = Right_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)

                Term1 += ( vel[dim-jj-1][indx[-1], indx[-2], indx[-3]].reshape(1,-1) \
                        * ( (np.squeeze(Left_terms_term1, axis=1) @ Mid_term_term1).reshape(-1,rm2) @ np.squeeze(Right_terms_term1, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term1, axis=1) @ Mid_term_F_term1).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term1, axis=2).T ) )  ).reshape(d1,d2,d3)
                
                Term2 += ( Nu_derivation[dim-jj-1][indx[-1], indx[-2], indx[-3]].reshape(1,-1) \
                       * ( (np.squeeze(Left_terms_term21, axis=1) @ Mid_term_term21).reshape(-1,rm2) @ np.squeeze(Right_terms_term21, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term21, axis=1) @ Mid_term_F_term21).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term21, axis=2).T  ) ) \
                        + Nu[indx[-1], indx[-2], indx[-3]].reshape(1,-1) \
                        * ( (np.squeeze(Left_terms_term22, axis=1) @ Mid_term_term22).reshape(-1,rm2) @ np.squeeze(Right_terms_term22, axis=2).T \
                           + C_RK*dt* ( (np.squeeze(Left_terms_F_term22, axis=1) @ Mid_term_F_term22).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term22, axis=2).T  ) )   ).reshape(d1,d2,d3)
                
            
            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            Last_3G = (G0[-3][:,indx[-3],:] @ (G0[-2][:,indx[-2],:] @ G0[-1][:,indx[-1],:].reshape(G0[-1].shape[0], -1)).reshape(G0[-2].shape[0], -1)).reshape(G0[-3].shape[0],-1)
            Last_3G_F = (G0_F[-3][:,indx[-3],:] @ (G0_F[-2][:,indx[-2],:] @ G0_F[-1][:,indx[-1],:].reshape(G0_F[-1].shape[0], -1)).reshape(G0_F[-2].shape[0], -1)).reshape(G0_F[-3].shape[0],-1)
            for jj in range(0,dim-3):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = Last_3G[:]+0;   Avg_Phi_F = Last_3G_F[:]+0
                for ii in reversed(range(0,dim-3)):
                    Avg_Phi = np.sum(Delta_Phi*New_G0_avg[ii], axis=1) @ Avg_Phi 
                    Avg_Phi_F = np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1) @ Avg_Phi_F
      
                Avg_Phi = Avg_Phi.reshape(len(indx[-3]), len(indx[-2]), len(indx[-1]))
                Avg_Phi_F = Avg_Phi_F.reshape(len(indx[-3]), len(indx[-2]), len(indx[-1]))
                Avg_idx = indx.copy()
                Avg_idx[-3] = np.arange(len(indx[-3]));   Avg_idx[-2] = np.arange(len(indx[-2]));   Avg_idx[-1] = np.arange(len(indx[-1]))

                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])       # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]       # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj] 
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0[jj]) 
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0_F[jj])

                Left_terms_term31 = np.moveaxis(New_G0_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term31 = np.moveaxis(New_G0_F_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_term32 = np.moveaxis(New_G0_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term32 = np.moveaxis(New_G0_F_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_term4 = np.moveaxis(New_G0_term4[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term4 = np.moveaxis(New_G0_F_term4[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term31 = Left_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term31 = Left_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term32 = Left_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term32 = Left_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term4 = Left_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term4 = Left_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(rm1, -1)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(rm_F1, -1)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(rm1, -1)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(rm_F1, -1)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(rm1, -1)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(rm_F1, -1)

                Right_terms_term31 = np.moveaxis(New_G0_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term31 = np.moveaxis(New_G0_F_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term32 = np.moveaxis(New_G0_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term32 = np.moveaxis(New_G0_F_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term4 = np.moveaxis(New_G0_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term4 = np.moveaxis(New_G0_F_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                    
                for ii in range(mode+2, dim):
                    Right_terms_term31 = Right_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term31 = Right_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term32 = Right_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term32 = Right_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term4 = Right_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term4 = Right_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)
                

                Term3 += ( Omega[indx[-1], indx[-2], indx[-3]].reshape(1,-1) \
                       * ( (np.squeeze(Left_terms_term31, axis=1) @ Mid_term_term31).reshape(-1,rm2) @ np.squeeze(Right_terms_term31, axis=2).T \
                          - Avg_Phi[Avg_idx[-3],Avg_idx[-2],Avg_idx[-1]].reshape(1,-1) * ( (np.squeeze(Left_terms_term32, axis=1) @ Mid_term_term32).reshape(-1,rm2) @ np.squeeze(Right_terms_term32, axis=2).T ) \
                        + C_RK*dt* ( (np.squeeze(Left_terms_F_term31, axis=1) @ Mid_term_F_term31).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term31, axis=2).T \
                                    - Avg_Phi_F[Avg_idx[-3],Avg_idx[-2],Avg_idx[-1]].reshape(1,-1) * ( (np.squeeze(Left_terms_F_term32, axis=1) @ Mid_term_F_term32).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term32, axis=2).T )  )  )   ).reshape(d1,d2,d3)     
                           
                Term4 += ( (np.squeeze(Left_terms_term4, axis=1) @ Mid_term_term4).reshape(-1,rm2) @ np.squeeze(Right_terms_term4, axis=2).T \
                         + C_RK*dt* ( (np.squeeze(Left_terms_F_term4, axis=1) @ Mid_term_F_term4).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term4, axis=2).T )   ).reshape(d1,d2,d3)
                        
            F = -1*Term1 + Term2 + Term3 - Term4

        

        if mode == dim-3:

            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm1 = G0[mode].shape[0]
            rm2 = G0[mode].shape[2]
            rm_F1 = G0_F[mode].shape[0]
            rm_F2 = G0_F[mode].shape[2]
            for jj in range(dim-3, dim):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj] 
                New_G0_F_term1[jj] = D1x @ G0_F[jj] 
                New_G0_term21[jj] = D1x @ G0[jj] 
                New_G0_F_term21[jj] = D1x @ G0_F[jj] 
                New_G0_term22[jj] = D2x @ G0[jj] 
                New_G0_F_term22[jj] = D2x @ G0_F[jj]

                Left_terms_term1 = np.moveaxis(New_G0_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term1 = np.moveaxis(New_G0_F_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_term21 = np.moveaxis(New_G0_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term21 = np.moveaxis(New_G0_F_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_term22 = np.moveaxis(New_G0_term22[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term22 = np.moveaxis(New_G0_F_term22[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term1 = Left_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term1 = Left_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term21 = Left_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term21 = Left_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term22 = Left_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term22 = Left_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(rm1, -1)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(rm_F1, -1)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(rm1, -1)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(rm_F1, -1)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(rm1, -1)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(rm_F1, -1)

                Right_terms_term1 = np.moveaxis(New_G0_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term1 = np.moveaxis(New_G0_F_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term21 = np.moveaxis(New_G0_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term21 = np.moveaxis(New_G0_F_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term22 = np.moveaxis(New_G0_term22[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term22 = np.moveaxis(New_G0_F_term22[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2,dim):
                    Right_terms_term1 = Right_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term1 = Right_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term21 = Right_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term21 = Right_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term22 = Right_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term22 = Right_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)


                Term1 += ( vel[dim-jj-1][indx[-1].reshape(1,-1), indx[-2].reshape(1,-1), indx[-3].reshape(-1,1)].reshape(1,-1) \
                        * ( (np.squeeze(Left_terms_term1, axis=1) @ Mid_term_term1).reshape(-1,rm2) @ np.squeeze(Right_terms_term1, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term1, axis=1) @ Mid_term_F_term1).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term1, axis=2).T ) ).reshape(d1,-1)  ).reshape(d1,d2,d3)
                
                Term2 += ( Nu_derivation[dim-jj-1][indx[-1].reshape(1,-1), indx[-2].reshape(1,-1), indx[-3].reshape(-1,1)].reshape(1,-1) \
                       * ( (np.squeeze(Left_terms_term21, axis=1) @ Mid_term_term21).reshape(-1,rm2) @ np.squeeze(Right_terms_term21, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term21, axis=1) @ Mid_term_F_term21).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term21, axis=2).T  ) ).reshape(d1,-1) \
                        + Nu[indx[-1].reshape(1,-1), indx[-2].reshape(1,-1), indx[-3].reshape(-1,1)].reshape(1,-1) \
                        * ( (np.squeeze(Left_terms_term22, axis=1) @ Mid_term_term22).reshape(-1,rm2) @ np.squeeze(Right_terms_term22, axis=2).T \
                           + C_RK*dt* ( (np.squeeze(Left_terms_F_term22, axis=1) @ Mid_term_F_term22).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term22, axis=2).T  ) ).reshape(d1,-1)   ).reshape(d1,d2,d3)    
            
            
            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            Last_3G = (G0[-3][:,indx[-3],:] @ (G0[-2][:,indx[-2],:] @ G0[-1][:,indx[-1],:].reshape(G0[-1].shape[0], -1)).reshape(G0[-2].shape[0], -1)).reshape(G0[-3].shape[0],-1)
            Last_3G_F = (G0_F[-3][:,indx[-3],:] @ (G0_F[-2][:,indx[-2],:] @ G0_F[-1][:,indx[-1],:].reshape(G0_F[-1].shape[0], -1)).reshape(G0_F[-2].shape[0], -1)).reshape(G0_F[-3].shape[0],-1)
            for jj in range(0,dim-3):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = Last_3G[:]+0;   Avg_Phi_F = Last_3G_F[:]+0
                for ii in reversed(range(0,dim-3)):
                    Avg_Phi = np.sum(Delta_Phi*New_G0_avg[ii], axis=1) @ Avg_Phi 
                    Avg_Phi_F = np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1) @ Avg_Phi_F
      
                Avg_Phi = Avg_Phi.reshape(N[-3], len(indx[-2]), len(indx[-1]))
                Avg_Phi_F = Avg_Phi_F.reshape(N[-3], len(indx[-2]), len(indx[-1]))
                Avg_idx = indx.copy()
                Avg_idx[-2] = np.arange(len(indx[-2]));   Avg_idx[-1] = np.arange(len(indx[-1]))

                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])       # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]     # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj] 
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0[jj])
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0_F[jj]) 

                Left_terms_term31 = np.moveaxis(New_G0_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term31 = np.moveaxis(New_G0_F_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_term32 = np.moveaxis(New_G0_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term32 = np.moveaxis(New_G0_F_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_term4 = np.moveaxis(New_G0_term4[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term4 = np.moveaxis(New_G0_F_term4[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term31 = Left_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term31 = Left_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term32 = Left_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term32 = Left_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term4 = Left_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term4 = Left_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(rm1, -1)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(rm_F1, -1)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(rm1, -1)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(rm_F1, -1)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(rm1, -1)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(rm_F1, -1)

                Right_terms_term31 = np.moveaxis(New_G0_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term31 = np.moveaxis(New_G0_F_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term32 = np.moveaxis(New_G0_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term32 = np.moveaxis(New_G0_F_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term4 = np.moveaxis(New_G0_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term4 = np.moveaxis(New_G0_F_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                    
                for ii in range(mode+2, dim):
                    Right_terms_term31 = Right_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term31 = Right_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term32 = Right_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term32 = Right_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term4 = Right_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term4 = Right_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)
                

                Term3 += ( Omega[indx[-1].reshape(1,-1), indx[-2].reshape(1,-1), indx[-3].reshape(-1,1)].reshape(1,-1) \
                       * ( ((np.squeeze(Left_terms_term31, axis=1) @ Mid_term_term31).reshape(-1,rm2) @ np.squeeze(Right_terms_term31, axis=2).T).reshape(d1,-1) \
                          - Avg_Phi[Avg_idx[-3].reshape(-1,1), Avg_idx[-2].reshape(1,-1), Avg_idx[-1].reshape(1,-1)].reshape(1,-1) * ( (np.squeeze(Left_terms_term32, axis=1) @ Mid_term_term32).reshape(-1,rm2) @ np.squeeze(Right_terms_term32, axis=2).T ).reshape(d1,-1) \
                        + C_RK*dt* ( ((np.squeeze(Left_terms_F_term31, axis=1) @ Mid_term_F_term31).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term31, axis=2).T).reshape(d1,-1) \
                                    - Avg_Phi_F[Avg_idx[-3].reshape(-1,1), Avg_idx[-2].reshape(1,-1), Avg_idx[-1].reshape(1,-1)].reshape(1,-1) * ( (np.squeeze(Left_terms_F_term32, axis=1) @ Mid_term_F_term32).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term32, axis=2).T ).reshape(d1,-1)  )  )   ).reshape(d1,d2,d3)     
                           
                Term4 += ( (np.squeeze(Left_terms_term4, axis=1) @ Mid_term_term4).reshape(-1,rm2) @ np.squeeze(Right_terms_term4, axis=2).T \
                         + C_RK*dt* ( (np.squeeze(Left_terms_F_term4, axis=1) @ Mid_term_F_term4).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term4, axis=2).T )   ).reshape(d1,d2,d3)
                        
                        
            F = -1*Term1 + Term2 + Term3 - Term4



        if mode == dim-2:

            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm1 = G0[mode].shape[0]
            rm2 = G0[mode].shape[2]
            rm_F1 = G0_F[mode].shape[0]
            rm_F2 = G0_F[mode].shape[2]
            for jj in range(dim-3, dim):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj] 
                New_G0_F_term1[jj] = D1x @ G0_F[jj] 
                New_G0_term21[jj] = D1x @ G0[jj] 
                New_G0_F_term21[jj] = D1x @ G0_F[jj] 
                New_G0_term22[jj] = D2x @ G0[jj] 
                New_G0_F_term22[jj] = D2x @ G0_F[jj]

                Left_terms_term1 = np.moveaxis(New_G0_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term1 = np.moveaxis(New_G0_F_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_term21 = np.moveaxis(New_G0_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term21 = np.moveaxis(New_G0_F_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_term22 = np.moveaxis(New_G0_term22[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term22 = np.moveaxis(New_G0_F_term22[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term1 = Left_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term1 = Left_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term21 = Left_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term21 = Left_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term22 = Left_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term22 = Left_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(rm1, -1)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(rm_F1, -1)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(rm1, -1)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(rm_F1, -1)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(rm1, -1)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(rm_F1, -1)

                Right_terms_term1 = np.moveaxis(New_G0_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term1 = np.moveaxis(New_G0_F_term1[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term21 = np.moveaxis(New_G0_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term21 = np.moveaxis(New_G0_F_term21[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term22 = np.moveaxis(New_G0_term22[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term22 = np.moveaxis(New_G0_F_term22[mode+1][:,New_idx[mode+1],:], 0, 1)

                for ii in range(mode+2,dim):
                    Right_terms_term1 = Right_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term1 = Right_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term21 = Right_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term21 = Right_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term22 = Right_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term22 = Right_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)


                Term1 += vel[dim-jj-1][indx[-1].reshape(1,1,-1), indx[-2].reshape(1,-1,1), indx[-3].reshape(-1,1,1)] \
                        * ( (np.squeeze(Left_terms_term1, axis=1) @ Mid_term_term1).reshape(-1,rm2) @ np.squeeze(Right_terms_term1, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term1, axis=1) @ Mid_term_F_term1).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term1, axis=2).T ) ).reshape(d1,d2,d3)
                
                Term2 +=  Nu_derivation[dim-jj-1][indx[-1].reshape(1,1,-1), indx[-2].reshape(1,-1,1), indx[-3].reshape(-1,1,1)] \
                       * ( (np.squeeze(Left_terms_term21, axis=1) @ Mid_term_term21).reshape(-1,rm2) @ np.squeeze(Right_terms_term21, axis=2).T \
                          + C_RK*dt* ( (np.squeeze(Left_terms_F_term21, axis=1) @ Mid_term_F_term21).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term21, axis=2).T  ) ).reshape(d1,d2,d3) \
                        + Nu[indx[-1].reshape(1,1,-1), indx[-2].reshape(1,-1,1), indx[-3].reshape(-1,1,1)] \
                        * ( (np.squeeze(Left_terms_term22, axis=1) @ Mid_term_term22).reshape(-1,rm2) @ np.squeeze(Right_terms_term22, axis=2).T \
                           + C_RK*dt* ( (np.squeeze(Left_terms_F_term22, axis=1) @ Mid_term_F_term22).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term22, axis=2).T  ) ).reshape(d1,d2,d3)   
            
            
            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            Last_3G = (G0[-3][:,indx[-3],:] @ (G0[-2][:,indx[-2],:] @ G0[-1][:,indx[-1],:].reshape(G0[-1].shape[0], -1)).reshape(G0[-2].shape[0], -1)).reshape(G0[-3].shape[0],-1)
            Last_3G_F = (G0_F[-3][:,indx[-3],:] @ (G0_F[-2][:,indx[-2],:] @ G0_F[-1][:,indx[-1],:].reshape(G0_F[-1].shape[0], -1)).reshape(G0_F[-2].shape[0], -1)).reshape(G0_F[-3].shape[0],-1)
            for jj in range(0,dim-3):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = Last_3G[:]+0;   Avg_Phi_F = Last_3G_F[:]+0
                for ii in reversed(range(0,dim-3)):
                    Avg_Phi = np.sum(Delta_Phi*New_G0_avg[ii], axis=1) @ Avg_Phi 
                    Avg_Phi_F = np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1) @ Avg_Phi_F
      
                Avg_Phi = Avg_Phi.reshape(len(indx[-3]), N[-2], len(indx[-1]))
                Avg_Phi_F = Avg_Phi_F.reshape(len(indx[-3]), N[-2], len(indx[-1]))
                Avg_idx = indx.copy()
                Avg_idx[-3] = np.arange(len(indx[-3]));   Avg_idx[-1] = np.arange(len(indx[-1]))

                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])       # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]     # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj] 
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0[jj]) 
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0_F[jj])

                Left_terms_term31 = np.moveaxis(New_G0_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term31 = np.moveaxis(New_G0_F_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_term32 = np.moveaxis(New_G0_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term32 = np.moveaxis(New_G0_F_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_term4 = np.moveaxis(New_G0_term4[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term4 = np.moveaxis(New_G0_F_term4[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term31 = Left_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term31 = Left_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term32 = Left_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term32 = Left_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term4 = Left_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term4 = Left_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(rm1, -1)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(rm_F1, -1)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(rm1, -1)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(rm_F1, -1)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(rm1, -1)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(rm_F1, -1)

                Right_terms_term31 = np.moveaxis(New_G0_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term31 = np.moveaxis(New_G0_F_term31[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term32 = np.moveaxis(New_G0_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term32 = np.moveaxis(New_G0_F_term32[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_term4 = np.moveaxis(New_G0_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                Right_terms_F_term4 = np.moveaxis(New_G0_F_term4[mode+1][:,New_idx[mode+1],:], 0, 1)
                    
                for ii in range(mode+2, dim):
                    Right_terms_term31 = Right_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term31 = Right_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term32 = Right_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term32 = Right_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_term4 = Right_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Right_terms_F_term4 = Right_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)
                

                Term3 += Omega[indx[-1].reshape(1,1,-1), indx[-2].reshape(1,-1,1), indx[-3].reshape(-1,1,1)] \
                       * ( ( (np.squeeze(Left_terms_term31, axis=1) @ Mid_term_term31).reshape(-1,rm2) @ np.squeeze(Right_terms_term31, axis=2).T ).reshape(d1,d2,d3) \
                          - Avg_Phi[Avg_idx[-3].reshape(-1,1,1), Avg_idx[-2].reshape(1,-1,1), Avg_idx[-1].reshape(1,1,-1)] * ( (np.squeeze(Left_terms_term32, axis=1) @ Mid_term_term32).reshape(-1,rm2) @ np.squeeze(Right_terms_term32, axis=2).T ).reshape(d1,d2,d3) \
                        + C_RK*dt* ( ( (np.squeeze(Left_terms_F_term31, axis=1) @ Mid_term_F_term31).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term31, axis=2).T ).reshape(d1,d2,d3) \
                                    - Avg_Phi_F[Avg_idx[-3].reshape(-1,1,1), Avg_idx[-2].reshape(1,-1,1), Avg_idx[-1].reshape(1,1,-1)] * ( (np.squeeze(Left_terms_F_term32, axis=1) @ Mid_term_F_term32).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term32, axis=2).T ).reshape(d1,d2,d3)  )  )      
                           
                Term4 += ( (np.squeeze(Left_terms_term4, axis=1) @ Mid_term_term4).reshape(-1,rm2) @ np.squeeze(Right_terms_term4, axis=2).T \
                         + C_RK*dt* ( (np.squeeze(Left_terms_F_term4, axis=1) @ Mid_term_F_term4).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F_term4, axis=2).T )   ).reshape(d1,d2,d3)


            F = -1*Term1 + Term2 + Term3 - Term4


    
        if mode == dim-1:

            Term1 = np.zeros((d1, d2, d3))
            Term2 = np.zeros((d1, d2, d3))
            rm1 = G0[mode].shape[0]
            rm2 = G0[mode].shape[2]
            rm_F1 = G0_F[mode].shape[0]
            rm_F2 = G0_F[mode].shape[2]
            for jj in range(dim-3, dim):
                New_G0_term1 = G0.copy();   New_G0_F_term1 = G0_F.copy()  # This term is v_i ∂p/∂x_i : Term 1 
                New_G0_term21 = G0.copy();   New_G0_F_term21 = G0_F.copy()  # This term is ∂Nu/∂x_i * ∂p/∂x_i  : Term21
                New_G0_term22 = G0.copy();   New_G0_F_term22 = G0_F.copy()  # This term is Nu* ∂2p/∂x_i^2  : Term22
                New_idx = indx.copy()

                New_G0_term1[jj] = D1x @ G0[jj] 
                New_G0_F_term1[jj] = D1x @ G0_F[jj] 
                New_G0_term21[jj] = D1x @ G0[jj] 
                New_G0_F_term21[jj] = D1x @ G0_F[jj] 
                New_G0_term22[jj] = D2x @ G0[jj] 
                New_G0_F_term22[jj] = D2x @ G0_F[jj]

                Left_terms_term1 = np.moveaxis(New_G0_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term1 = np.moveaxis(New_G0_F_term1[0][:,New_idx[0],:], 0, 1)
                Left_terms_term21 = np.moveaxis(New_G0_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term21 = np.moveaxis(New_G0_F_term21[0][:,New_idx[0],:], 0, 1)
                Left_terms_term22 = np.moveaxis(New_G0_term22[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term22 = np.moveaxis(New_G0_F_term22[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term1 = Left_terms_term1 @ np.moveaxis(New_G0_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term1 = Left_terms_F_term1 @ np.moveaxis(New_G0_F_term1[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term21 = Left_terms_term21 @ np.moveaxis(New_G0_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term21 = Left_terms_F_term21 @ np.moveaxis(New_G0_F_term21[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term22 = Left_terms_term22 @ np.moveaxis(New_G0_term22[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term22 = Left_terms_F_term22 @ np.moveaxis(New_G0_F_term22[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term1 = (New_G0_term1[mode][:]).reshape(rm1, -1)
                Mid_term_F_term1 = (New_G0_F_term1[mode][:]).reshape(rm_F1, -1)
                Mid_term_term21 = (New_G0_term21[mode][:]).reshape(rm1, -1)
                Mid_term_F_term21 = (New_G0_F_term21[mode][:]).reshape(rm_F1, -1)
                Mid_term_term22 = (New_G0_term22[mode][:]).reshape(rm1, -1)
                Mid_term_F_term22 = (New_G0_F_term22[mode][:]).reshape(rm_F1, -1)

                
                Term1 += ( vel[dim-jj-1][indx[-1].reshape(1,-1), indx[-2].reshape(-1,1), indx[-3].reshape(-1,1)] \
                       * ( np.squeeze(Left_terms_term1, axis=1) @ Mid_term_term1 \
                          + C_RK*dt* ( np.squeeze(Left_terms_F_term1, axis=1) @ Mid_term_F_term1 ) )  ).reshape(d1,d2,d3)
                
                Term2 += ( Nu_derivation[dim-jj-1][indx[-1].reshape(1,-1), indx[-2].reshape(-1,1), indx[-3].reshape(-1,1)]  \
                       * ( np.squeeze(Left_terms_term21, axis=1) @ Mid_term_term21 \
                          + C_RK*dt* ( np.squeeze(Left_terms_F_term21, axis=1) @ Mid_term_F_term21 ) ) \
                        + Nu[indx[-1].reshape(1,-1), indx[-2].reshape(-1,1), indx[-3].reshape(-1,1)] \
                        * ( np.squeeze(Left_terms_term22, axis=1) @ Mid_term_term22 \
                           + C_RK*dt* ( np.squeeze(Left_terms_F_term22, axis=1) @ Mid_term_F_term22 ) )  ).reshape(d1,d2,d3)
                
            
            Term3 = np.zeros((d1, d2, d3))
            Term4 = np.zeros((d1, d2, d3))
            Last_3G = (G0[-3][:,indx[-3],:] @ (G0[-2][:,indx[-2],:] @ G0[-1][:,indx[-1],:].reshape(G0[-1].shape[0], -1)).reshape(G0[-2].shape[0], -1)).reshape(G0[-3].shape[0],-1)
            Last_3G_F = (G0_F[-3][:,indx[-3],:] @ (G0_F[-2][:,indx[-2],:] @ G0_F[-1][:,indx[-1],:].reshape(G0_F[-1].shape[0], -1)).reshape(G0_F[-2].shape[0], -1)).reshape(G0_F[-3].shape[0],-1)
            for jj in range(0,dim-3):
                New_G0_avg = G0.copy();   New_G0_F_avg = G0_F.copy()
                New_G0_avg[jj] = Phi*G0[jj];    New_G0_F_avg[jj] = Phi*G0_F[jj]
                Avg_Phi = Last_3G[:]+0;   Avg_Phi_F = Last_3G_F[:]+0
                for ii in reversed(range(0,dim-3)):
                    Avg_Phi = np.sum(Delta_Phi*New_G0_avg[ii], axis=1) @ Avg_Phi 
                    Avg_Phi_F = np.sum(Delta_Phi*New_G0_F_avg[ii], axis=1) @ Avg_Phi_F
      
                Avg_Phi = Avg_Phi.reshape(len(indx[-3]), len(indx[-2]), N[-1])
                Avg_Phi_F = Avg_Phi_F.reshape(len(indx[-3]), len(indx[-2]), N[-1])
                Avg_idx = indx.copy()
                Avg_idx[-3] = np.arange(len(indx[-3]));   Avg_idx[-2] = np.arange(len(indx[-2]));

                New_G0_term31 = G0.copy();   New_G0_F_term31 = G0_F.copy()
                New_G0_term32 = G0.copy();   New_G0_F_term32 = G0_F.copy()
                New_G0_term4 = G0.copy();   New_G0_F_term4 = G0_F.copy()
                New_idx = indx.copy()
                
                New_G0_term31[jj] = D1_Phi @ (Phi * G0[jj])       # φ ∂p/∂φ
                New_G0_F_term31[jj] = D1_Phi @ (Phi * G0_F[jj]) 
                New_G0_term32[jj] = D1_Phi @ G0[jj]     # ∂p/∂φ
                New_G0_F_term32[jj] = D1_Phi @ G0_F[jj]
                New_G0_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0[jj]) 
                New_G0_F_term4[jj] = D1_Phi @ (S_alpha[:,-(jj+1)].reshape(-1,1)*G0_F[jj])

                Left_terms_term31 = np.moveaxis(New_G0_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term31 = np.moveaxis(New_G0_F_term31[0][:,New_idx[0],:], 0, 1)
                Left_terms_term32 = np.moveaxis(New_G0_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term32 = np.moveaxis(New_G0_F_term32[0][:,New_idx[0],:], 0, 1)
                Left_terms_term4 = np.moveaxis(New_G0_term4[0][:,New_idx[0],:], 0, 1)
                Left_terms_F_term4 = np.moveaxis(New_G0_F_term4[0][:,New_idx[0],:], 0, 1)

                for ii in range(1, mode):
                    Left_terms_term31 = Left_terms_term31 @ np.moveaxis(New_G0_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term31 = Left_terms_F_term31 @ np.moveaxis(New_G0_F_term31[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term32 = Left_terms_term32 @ np.moveaxis(New_G0_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term32 = Left_terms_F_term32 @ np.moveaxis(New_G0_F_term32[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_term4 = Left_terms_term4 @ np.moveaxis(New_G0_term4[ii][:,New_idx[ii],:], 0, 1)
                    Left_terms_F_term4 = Left_terms_F_term4 @ np.moveaxis(New_G0_F_term4[ii][:,New_idx[ii],:], 0, 1)

                Mid_term_term31 = (New_G0_term31[mode][:]).reshape(rm1, -1)
                Mid_term_F_term31 = (New_G0_F_term31[mode][:]).reshape(rm_F1, -1)
                Mid_term_term32 = (New_G0_term32[mode][:]).reshape(rm1, -1)
                Mid_term_F_term32 = (New_G0_F_term32[mode][:]).reshape(rm_F1, -1)
                Mid_term_term4 = (New_G0_term4[mode][:]).reshape(rm1, -1)
                Mid_term_F_term4 = (New_G0_F_term4[mode][:]).reshape(rm_F1, -1) 

            
                Term3 += ( Omega[indx[-1].reshape(1,-1), indx[-2].reshape(-1,1), indx[-3].reshape(-1,1)] \
                       * ( np.squeeze(Left_terms_term31, axis=1) @ Mid_term_term31 \
                          - Avg_Phi[Avg_idx[-3].reshape(-1,1), Avg_idx[-2].reshape(-1,1), Avg_idx[-1].reshape(1,-1)] * ( np.squeeze(Left_terms_term32, axis=1) @ Mid_term_term32 ) \
                          + C_RK*dt* ( np.squeeze(Left_terms_F_term31, axis=1) @ Mid_term_F_term31 \
                                    - Avg_Phi_F[Avg_idx[-3].reshape(-1,1), Avg_idx[-2].reshape(-1,1), Avg_idx[-1].reshape(1,-1)] * ( np.squeeze(Left_terms_F_term32, axis=1) @ Mid_term_F_term32)  )  )   ).reshape(d1,d2,d3)
                                 
                Term4 += ( np.squeeze(Left_terms_term4, axis=1) @ Mid_term_term4 \
                         + C_RK*dt* ( np.squeeze(Left_terms_F_term4, axis=1) @ Mid_term_F_term4 ) ).reshape(d1,d2,d3)
                        
                        
            F = -1*Term1 + Term2 + Term3 - Term4

    return F

# ============================================================================================================
# A function to approximate the Right-Hand side of the PDE in tensor train format using TT-CUR-DEIM

def Approximate_RHS(iter_count, Adaptivity, C_RK, G0_F, G0, J_F):
 
    G_F =[]
    Next_idx = []
    deim_idx = []

    if Adaptivity =='OFF':
    
            temp_idx = []
            temp_idx.append(np.arange(N[0]));    temp_idx.extend(J_F[0:dim-1])
            C_first = np.squeeze(Eval_RHS(iter_count, 0, C_RK, G0_F, G0, temp_idx, 1, N[0], len(temp_idx[1])), axis=0)
            U_first,_,_ = np.linalg.svd(C_first, full_matrices=False)

            Next_idx.append(myDEIM(U_first, r[0]))
            G_F.append((U_first[:, 0:r[0]] @ np.linalg.pinv(U_first[Next_idx[0], 0:r[0]])).reshape(1, N[0], r[0]))

            for z in range(1,dim-1):
                temp_idx = []
                temp_idx.extend(Next_idx[-z:]);     temp_idx.append(np.arange(N[z]));      temp_idx.extend(J_F[int(z*dim-0.5*(z**2+z)) : int((z+1)*dim-0.5*(z**2+3*z+2))])
                C_mid = Eval_RHS(iter_count, z, C_RK, G0_F, G0, temp_idx, r[z-1], N[z], len(temp_idx[z+1]))
                C_mid = C_mid.reshape(r[z-1]*N[z], -1)
                U_mid,_,_ = np.linalg.svd(C_mid, full_matrices=False)

                I_combined = myDEIM(U_mid, r[z])
                [[Next_idx.append(np.array(Next_idx[int((z*(z-1))/2+ln)][np.array(np.floor(I_combined/N[z]), int)], int))] for ln in range(z)]
                Next_idx.append(np.array(I_combined - N[z]*np.floor(I_combined/N[z]), int))
                G_F.append((U_mid[:, 0:r[z]] @ np.linalg.pinv(U_mid[I_combined, 0:r[z]])).reshape(r[z-1],N[z],r[z]))

            
            temp_idx = []
            temp_idx.extend(Next_idx[-(dim-1):]);    temp_idx.append(np.arange(N[dim-1]))
            C_end = Eval_RHS(iter_count, dim-1, C_RK, G0_F, G0, temp_idx, r[-1], N[dim-1], 1)
            G_F.append(C_end)


    if Adaptivity =='ON':
            
            temp_idx = []
            temp_idx.append(np.arange(N[0]));    temp_idx.extend(J_F[0:dim-1])
            C_first = np.squeeze(Eval_RHS(iter_count, 0, C_RK, G0_F, G0, temp_idx, 1, N[0], len(temp_idx[1])), axis=0)
            U_first,_,_ = np.linalg.svd(C_first, full_matrices=False)            
            I_temp = myDEIM(U_first[:, 0:r[0]], r[0])
            deim_idx.append(I_temp)
            I_temp = GappyPOD(U_first[:, 0:r[0]], N_os, I_temp)
            Next_idx.append(I_temp)

            for z in range(1,dim-1):
                temp_idx = []
                temp_idx.extend(deim_idx[-z:]);     temp_idx.append(np.arange(N[z]));      temp_idx.extend(J_F[int(z*dim-0.5*(z**2+z)) : int((z+1)*dim-0.5*(z**2+3*z+2))])
                C_mid = Eval_RHS(iter_count, z, C_RK, G0_F, G0, temp_idx, r[z-1], N[z], len(temp_idx[z+1]))
                C_mid = C_mid.reshape(r[z-1]*N[z], -1)
                U_mid,_,_ = np.linalg.svd(C_mid, full_matrices=False)
                I_combined = myDEIM(U_mid[:, 0:r[z]], r[z])
                [[deim_idx.append(np.array(deim_idx[int((z*(z-1))/2+ln)][np.array(np.floor(I_combined/N[z]), int)], int))]  for ln in range(z)]
                deim_idx.append(np.array(I_combined - N[z]*np.floor(I_combined/N[z]), int))
                I_combined = GappyPOD(U_mid[:, 0:r[z]], N_os, I_combined)
                [[Next_idx.append(np.array(Next_idx[int((z*(z-1))/2+ln)][np.array(np.floor(I_combined/N[z]), int)], int))]  for ln in range(z)]
                Next_idx.append(np.array(I_combined - N[z]*np.floor(I_combined/N[z]), int))

    
    Next_idx = Next_idx[::-1]

    return G_F, Next_idx

# ============================================================================================================
# A function to evaluate the fibers of V in tensor train format using TT-CUR-DEIM

def Fiber_eval(G0, G1_F, G2_F, G3_F, G4_F, indx, mode, d1, d2, d3):

    V_hat =  np.zeros((d1, d2, d3))
    F1 = np.zeros((d1, d2, d3))
    F2 = np.zeros((d1, d2, d3))
    F3 = np.zeros((d1, d2, d3))
    F4 = np.zeros((d1, d2, d3))

    if mode == 0:
        rm = G0[mode].shape[2]
        rm_F = G1_F[mode].shape[2]

        Mid_term = (G0[mode][:]).reshape(-1, rm)
        Mid_term_F1 = (G1_F[mode][:]).reshape(-1, rm_F)
        Mid_term_F2 = (G2_F[mode][:]).reshape(-1, rm_F)
        Mid_term_F3 = (G3_F[mode][:]).reshape(-1, rm_F)
        Mid_term_F4 = (G4_F[mode][:]).reshape(-1, rm_F)

        Right_terms = np.moveaxis(G0[mode+1][:,indx[mode+1],:], 0, 1)
        Right_terms_F1 = np.moveaxis(G1_F[mode+1][:,indx[mode+1],:], 0, 1)
        Right_terms_F2 = np.moveaxis(G2_F[mode+1][:,indx[mode+1],:], 0, 1)
        Right_terms_F3 = np.moveaxis(G3_F[mode+1][:,indx[mode+1],:], 0, 1)
        Right_terms_F4 = np.moveaxis(G4_F[mode+1][:,indx[mode+1],:], 0, 1)

        for ii in range(mode+2,dim):
            Right_terms = Right_terms @ np.moveaxis(G0[ii][:,indx[ii],:], 0, 1)
            Right_terms_F1 = Right_terms_F1 @ np.moveaxis(G1_F[ii][:,indx[ii],:], 0, 1)
            Right_terms_F2 = Right_terms_F2 @ np.moveaxis(G2_F[ii][:,indx[ii],:], 0, 1)
            Right_terms_F3 = Right_terms_F3 @ np.moveaxis(G3_F[ii][:,indx[ii],:], 0, 1)
            Right_terms_F4 = Right_terms_F4 @ np.moveaxis(G4_F[ii][:,indx[ii],:], 0, 1)

        V_hat = (Mid_term @ np.squeeze(Right_terms, axis=2).T).reshape(d1,d2,d3)
        F1 = (Mid_term_F1 @ np.squeeze(Right_terms_F1, axis=2).T).reshape(d1,d2,d3)
        F2 = (Mid_term_F2 @ np.squeeze(Right_terms_F2, axis=2).T).reshape(d1,d2,d3)
        F3 = (Mid_term_F3 @ np.squeeze(Right_terms_F3, axis=2).T).reshape(d1,d2,d3)
        F4 = (Mid_term_F4 @ np.squeeze(Right_terms_F4, axis=2).T).reshape(d1,d2,d3)


    if mode == dim-1:
        rm = G0[mode].shape[0]
        rm_F = G1_F[mode].shape[0]

        Left_terms = np.moveaxis(G0[0][:,indx[0],:], 0, 1)
        Left_terms_F1 = np.moveaxis(G1_F[0][:,indx[0],:], 0, 1)
        Left_terms_F2 = np.moveaxis(G2_F[0][:,indx[0],:], 0, 1)
        Left_terms_F3 = np.moveaxis(G3_F[0][:,indx[0],:], 0, 1)
        Left_terms_F4 = np.moveaxis(G4_F[0][:,indx[0],:], 0, 1)

        for ii in range(1,mode):
            Left_terms = Left_terms @ np.moveaxis(G0[ii][:,indx[ii],:], 0, 1)
            Left_terms_F1 = Left_terms_F1 @ np.moveaxis(G1_F[ii][:,indx[ii],:], 0, 1)
            Left_terms_F2 = Left_terms_F2 @ np.moveaxis(G2_F[ii][:,indx[ii],:], 0, 1)
            Left_terms_F3 = Left_terms_F3 @ np.moveaxis(G3_F[ii][:,indx[ii],:], 0, 1)
            Left_terms_F4 = Left_terms_F4 @ np.moveaxis(G4_F[ii][:,indx[ii],:], 0, 1) 

        Mid_term = (G0[mode][:]).reshape(rm, -1)
        Mid_term_F1 = (G1_F[mode][:]).reshape(rm_F, -1)
        Mid_term_F2 = (G2_F[mode][:]).reshape(rm_F, -1)
        Mid_term_F3 = (G3_F[mode][:]).reshape(rm_F, -1)
        Mid_term_F4 = (G4_F[mode][:]).reshape(rm_F, -1)


        V_hat = (np.squeeze(Left_terms, axis=1) @ Mid_term).reshape(d1,d2,d3) 
        F1 = (np.squeeze(Left_terms_F1, axis=1) @ Mid_term_F1).reshape(d1,d2,d3) 
        F2 = (np.squeeze(Left_terms_F2, axis=1) @ Mid_term_F2).reshape(d1,d2,d3) 
        F3 = (np.squeeze(Left_terms_F3, axis=1) @ Mid_term_F3).reshape(d1,d2,d3) 
        F4 = (np.squeeze(Left_terms_F4, axis=1) @ Mid_term_F4).reshape(d1,d2,d3) 
        

    elif mode !=0 and mode!= dim-1:
        rm1 = G0[mode].shape[0]
        rm2 = G0[mode].shape[2]
        rm_F1 = G1_F[mode].shape[0]
        rm_F2 = G1_F[mode].shape[2]

        Left_terms = np.moveaxis(G0[0][:,indx[0],:], 0, 1)
        Left_terms_F1 = np.moveaxis(G1_F[0][:,indx[0],:], 0, 1)
        Left_terms_F2 = np.moveaxis(G2_F[0][:,indx[0],:], 0, 1)
        Left_terms_F3 = np.moveaxis(G3_F[0][:,indx[0],:], 0, 1)
        Left_terms_F4 = np.moveaxis(G4_F[0][:,indx[0],:], 0, 1)

        for ii in range(1,mode):
            Left_terms = Left_terms @ np.moveaxis(G0[ii][:,indx[ii],:], 0, 1)
            Left_terms_F1 = Left_terms_F1 @ np.moveaxis(G1_F[ii][:,indx[ii],:], 0, 1)
            Left_terms_F2 = Left_terms_F2 @ np.moveaxis(G2_F[ii][:,indx[ii],:], 0, 1)
            Left_terms_F3 = Left_terms_F3 @ np.moveaxis(G3_F[ii][:,indx[ii],:], 0, 1)
            Left_terms_F4 = Left_terms_F4 @ np.moveaxis(G4_F[ii][:,indx[ii],:], 0, 1) 

        Mid_term = (G0[mode][:]).reshape(rm1, -1)
        Mid_term_F1 = (G1_F[mode][:]).reshape(rm_F1, -1)
        Mid_term_F2 = (G2_F[mode][:]).reshape(rm_F1, -1)
        Mid_term_F3 = (G3_F[mode][:]).reshape(rm_F1, -1)
        Mid_term_F4 = (G4_F[mode][:]).reshape(rm_F1, -1)

        Right_terms = np.moveaxis(G0[mode+1][:,indx[mode+1],:], 0, 1)
        Right_terms_F1 = np.moveaxis(G1_F[mode+1][:,indx[mode+1],:], 0, 1)
        Right_terms_F2 = np.moveaxis(G2_F[mode+1][:,indx[mode+1],:], 0, 1)
        Right_terms_F3 = np.moveaxis(G3_F[mode+1][:,indx[mode+1],:], 0, 1)
        Right_terms_F4 = np.moveaxis(G4_F[mode+1][:,indx[mode+1],:], 0, 1)

        for ii in range(mode+2,dim):
            Right_terms = Right_terms @ np.moveaxis(G0[ii][:,indx[ii],:], 0, 1)
            Right_terms_F1 = Right_terms_F1 @ np.moveaxis(G1_F[ii][:,indx[ii],:], 0, 1)
            Right_terms_F2 = Right_terms_F2 @ np.moveaxis(G2_F[ii][:,indx[ii],:], 0, 1)
            Right_terms_F3 = Right_terms_F3 @ np.moveaxis(G3_F[ii][:,indx[ii],:], 0, 1)
            Right_terms_F4 = Right_terms_F4 @ np.moveaxis(G4_F[ii][:,indx[ii],:], 0, 1)

        V_hat = ( (np.squeeze(Left_terms, axis=1) @ Mid_term).reshape(-1,rm2) @ np.squeeze(Right_terms, axis=2).T ).reshape(d1,d2,d3)
        F1 = ( (np.squeeze(Left_terms_F1, axis=1) @ Mid_term_F1).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F1, axis=2).T ).reshape(d1,d2,d3) 
        F2 = ( (np.squeeze(Left_terms_F2, axis=1) @ Mid_term_F2).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F2, axis=2).T ).reshape(d1,d2,d3) 
        F3 = ( (np.squeeze(Left_terms_F3, axis=1) @ Mid_term_F3).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F3, axis=2).T ).reshape(d1,d2,d3) 
        F4 = ( (np.squeeze(Left_terms_F4, axis=1) @ Mid_term_F4).reshape(-1,rm_F2) @ np.squeeze(Right_terms_F4, axis=2).T ).reshape(d1,d2,d3) 


    VvV = V_hat + dt*(F1 + 2.0*F2 + 2.0*F3 + F4) / 6

    return VvV


# ============================================================================================================
# Calculates the G0, G1, G2, ... at t=0

G1, G0, J0 = TT_DEIM(r,dim, max_iteration)

r = r[::-1]
J_F = J0[:]

# ============================================================================================================
# The main loop for time advancement

total_steps = round(Total_time / dt)
times= []
All_r = np.zeros((total_steps+2, dim-1))
Mean_Error = np.zeros((total_steps+2, 4))
Mean_of_mean = np.zeros((total_steps+2, 2))

intg_idx = np.random.randint(0, Nx[0], (3,200))

for p in range(total_steps+1):

    All_S = []

    G0_F = []  # Cores containing zero elements for the first iteration of the Runge-Kutta scheme 
    G0_F.append(np.zeros((1, N[0], G0[0].shape[2])));      [G0_F.append(np.zeros((G0[kj].shape[0],N[kj],G0[kj].shape[2])))  for kj in range(1, dim-1)];      G0_F.append(np.zeros((G0[dim-1].shape[0], N[dim-1], 1)))
# ...........................................................................................................................
    G1_F, Next_idx = Approximate_RHS(p, 'OFF', 0, G0_F, G0, J_F)
    G2_F, _= Approximate_RHS(p, 'OFF', 0.5, G1_F, G0, J_F)
    G3_F, _ = Approximate_RHS(p, 'OFF', 0.5, G2_F, G0, J_F)
    G4_F, _ = Approximate_RHS(p, 'OFF',  1 , G3_F, G0, J_F)

    # ...........................................................................................................................
    I = []
    Cores_current = []
    Cores_next = []

    temp_idx = []
    temp_idx.append(np.arange(N[0]));    temp_idx.extend(J0[0:dim-1]);
    V_first  = np.squeeze(Fiber_eval(G0, G1_F, G2_F, G3_F, G4_F, temp_idx, 0, 1, N[0], len(temp_idx[1])), axis=0)
    U_first,S_first,_ = np.linalg.svd(V_first, full_matrices=False)
    All_S.append(S_first[0:r[0]])
    I.append(myDEIM(U_first, r[0]))
    G = (U_first[:, 0:r[0]] @ np.linalg.pinv(U_first[I[0], 0:r[0]])).reshape(1,N[0],r[0]) # Determines the first Core tensor of size 1*N*r
    Cores_current.append(G)   
    Cores_next.append(np.moveaxis(G, [0,2], [2,0]))   # Transpose the core for the next time step

    for z in range(1, dim-1):
        temp_idx = []
        temp_idx.extend(I[-z:]);   temp_idx.append(np.arange(N[z]));    temp_idx.extend(J0[int(z*dim-0.5*(z**2+z)) : int((z+1)*dim-0.5*(z**2+3*z+2))])
        V_mid  = Fiber_eval(G0, G1_F, G2_F, G3_F, G4_F, temp_idx, z, r[z-1], N[z], len(temp_idx[z+1]))
        V_mid = V_mid.reshape(r[z-1]*N[z], -1)
        U_mid,S_mid,_ = np.linalg.svd(V_mid, full_matrices=False)
        All_S.append(S_mid[0:r[z]])
        I_combined = myDEIM(U_mid, r[z])
        [[I.append(np.array(I[int((z*(z-1))/2+ln)][np.array(np.floor(I_combined/N[z]), int)], int))] for ln in range(z)]
        I.append(np.array(I_combined - N[z]*np.floor(I_combined/N[z]), int))
        G = (U_mid[:, 0:r[z]]@ np.linalg.pinv(U_mid[I_combined, 0:r[z]])).reshape(r[z-1],N[z],r[z]) # Determines the middle Core tensors of size rN*r
        Cores_current.append(G)
        Cores_next.append(np.moveaxis(G, [0,2], [2,0]))   # Flip the axis for the nest time step

    temp_idx = []
    temp_idx.extend(I[-(dim-1):]);   temp_idx.append(np.arange(N[dim-1]));
    V_end = Fiber_eval(G0, G1_F, G2_F, G3_F, G4_F, temp_idx, dim-1, r[-1], N[dim-1], 1)
    Cores_current.append(V_end)   # Determines the last Core tensor of size r*N*1
    Cores_next.append(np.moveaxis(V_end, [0,2], [2,0]))

    # ...........................................................................................................................
    # rank adaptivity section

    eps= np.zeros((dim-1))
    r_updated = r+0
    add_rank = 0
    for jg in range(dim-1):
        eps[jg] = All_S[jg][-1]/np.sqrt(np.sum(All_S[jg]**2))
    
        if eps[jg] > eps_U:
            if add_rank == 0:          
                I, Deim_I = [], []
                _ , Next_idx = Approximate_RHS(p, 'ON', 0, G0_F, G0, J_F)
                # ...........................................................................................................................
                U_first, _, _ = np.linalg.svd(V_first, full_matrices=False)
                I_temp = myDEIM(U_first[:, 0:r[0]], r[0])
                Deim_I.append(I_temp)
                I_temp = GappyPOD(U_first[:, 0:r[0]], N_os, I_temp)
                I.append(I_temp)
                # ...........................................................................................................................
                for z in range(1, dim-1):
                    temp_idx = []
                    temp_idx.extend(Deim_I[-z:]);   temp_idx.append(np.arange(N[z]));    temp_idx.extend(J0[int(z*dim-0.5*(z**2+z)) : int((z+1)*dim-0.5*(z**2+3*z+2))])
                    V_mid  = Fiber_eval(G0, G1_F, G2_F, G3_F, G4_F, temp_idx, z, r[z-1], N[z], len(temp_idx[z+1]))
                    V_mid = V_mid.reshape(r[z-1]*N[z], -1)
                    U_mid,_,_ = np.linalg.svd(V_mid, full_matrices=False)
                    I_combined = myDEIM(U_mid[:, 0:r[z]], r[z])
                    [[Deim_I.append(np.array(Deim_I[int((z*(z-1))/2+ln)][np.array(np.floor(I_combined/N[z]), int)], int))] for ln in range(z)]
                    Deim_I.append(np.array(I_combined - N[z]*np.floor(I_combined/N[z]), int))
                    I_combined = GappyPOD(U_mid[:, 0:r[z]], N_os, I_combined)
                    [[I.append(np.array(I[int((z*(z-1))/2+ln)][np.array(np.floor(I_combined/N[z]), int)], int))] for ln in range(z)]
                    I.append(np.array(I_combined - N[z]*np.floor(I_combined/N[z]), int))
                
                add_rank = 1
                # ...........................................................................................................................
                
            r_updated[jg] = r[jg]+1 

        
        elif eps[jg] < eps_L and r[jg] > 2.5:
                r_updated[jg] = r[jg]-1

    # ...........................................................................................................................    
    J0 = I[::-1]  # Flip left-nested indices and use them as right-nested indices in the reverse iteration
    N = N[::-1]   # Flip N to be used in the reverse iteration based on the correct order
    G0 = Cores_next[::-1]
    r = r_updated[::-1]+0
    All_D2 = All_D2[::-1]
    J_F = Next_idx[:]
    
    # ...........................................................................................................................
    # post-processing section. Computes some results and stores them at specific time-steps.
    
    print('current rank = ',r)
    times.append(p*dt)
    All_r[p,:] = r

    if np.remainder(p, 2)==1:
        All_r[p,:] = r[::-1]

    if np.remainder(p, 20)==0:
        integral = np.zeros((intg_idx.shape[1]))
        for iI in range(intg_idx.shape[1]):
            integral[iI] = Delta_Phi**2 * np.sum((np.squeeze(G0[0][:,np.array([intg_idx[0,iI]]),:], axis=1) @ (np.squeeze(G0[1][:,np.array([intg_idx[1,iI]]),:], axis=1) @ (np.squeeze(G0[2][:,np.array([intg_idx[2,iI]]),:], axis=1) @ (G0[3] @ np.squeeze(G0[4], axis=2)).reshape(-1,N[3]*N[4])))).reshape(N[3], N[4]))
        
        RMSE = np.sqrt( np.sum( (integral - 1)**2 ) / intg_idx.shape[1] )
        print("RMSE=", RMSE, "Max=", np.max(integral), "Min=", np.min(integral), "Mean=", np.mean(integral), "Std=", np.std(integral))
        Mean_Error[int(p/20),0] = RMSE*1
        Mean_Error[int(p/20),1] = np.max(integral)
        Mean_Error[int(p/20),2] = np.min(integral)
        Mean_Error[int(p/20),3] = np.mean(integral)
        
        avg_Phi_1 = ( (G0[0] @ (G0[1] @ G0[2].reshape(G0[2].shape[0], -1)).reshape(G0[1].shape[0], -1)).reshape(-1, G0[2].shape[2]) @ np.sum(Delta_Phi*Phi*G0[3], axis=1) @ np.sum(Delta_Phi*G0[4], axis=1) ).reshape(N[0], N[1], N[2])
        avg_Phi_2 = ( (G0[0] @ (G0[1] @ G0[2].reshape(G0[2].shape[0], -1)).reshape(G0[1].shape[0], -1)).reshape(-1, G0[2].shape[2]) @ np.sum(Delta_Phi*G0[3], axis=1) @ np.sum(Delta_Phi*Phi*G0[4], axis=1) ).reshape(N[0], N[1], N[2])

        Mean_of_mean[int(p/20),0] = np.mean(avg_Phi_1)
        Mean_of_mean[int(p/20),1] = np.mean(avg_Phi_2)
        print("Mean_of_mean_Phi1=", Mean_of_mean[int(p/20),0], "Mean_of_mean_Phi2=", Mean_of_mean[int(p/20),1])

    print('current time = ', p*dt)
    # ...........................................................................................................................


# ============================================================================================================
# Save and plot the results

df1 = pd.DataFrame(Mean_Error)
df2 = pd.DataFrame(All_r)
df3 = pd.DataFrame(Mean_of_mean)
writer = pd.ExcelWriter("PDFTransportRanks.xlsx", engine='xlsxwriter')
df1.to_excel(writer, sheet_name='Errors', index=False)
df2.to_excel(writer, sheet_name='All_r', index=False)
df3.to_excel(writer, sheet_name='RANS_Mean', index=False)
writer.close()


P_a = (np.squeeze(G0[0][:,np.array([28]),:], axis=1) @ (np.squeeze(G0[1][:,np.array([28]),:], axis=1) @ (np.squeeze(G0[2][:,np.array([28]),:], axis=1) @ (G0[3] @ np.squeeze(G0[4], axis=2)).reshape(-1,N[3]*N[4])))).reshape(N[3], N[4])
P_b = (np.squeeze(G0[0][:,np.array([45]),:], axis=1) @ (np.squeeze(G0[1][:,np.array([45]),:], axis=1) @ (np.squeeze(G0[2][:,np.array([45]),:], axis=1) @ (G0[3] @ np.squeeze(G0[4], axis=2)).reshape(-1,N[3]*N[4])))).reshape(N[3], N[4])
P_c = (np.squeeze(G0[0][:,np.array([60]),:], axis=1) @ (np.squeeze(G0[1][:,np.array([60]),:], axis=1) @ (np.squeeze(G0[2][:,np.array([60]),:], axis=1) @ (G0[3] @ np.squeeze(G0[4], axis=2)).reshape(-1,N[3]*N[4])))).reshape(N[3], N[4])

P_d = ((np.squeeze(G0[0], axis=0) @ G0[1].reshape(G0[1].shape[0], -1)).reshape(N[0]*N[1], -1) @ np.squeeze(G0[2][:,np.array([20]),:], axis=1) @ (np.squeeze(G0[3][:, np.array([36]), :], axis=1) @ np.squeeze(G0[4][:, np.array([54])], axis=1))).reshape(N[0],N[1])
P_e = ((np.squeeze(G0[0], axis=0) @ (np.squeeze(G0[1][:,np.array([50]),:], axis=1)) @ G0[2].reshape(G0[2].shape[0], -1)).reshape(N[0]*N[2], -1) @ (np.squeeze(G0[3][:, np.array([36]), :], axis=1) @ np.squeeze(G0[4][:, np.array([54])], axis=1))).reshape(N[0],N[1])
P_f = (((np.squeeze(G0[0][:,np.array([70]),:], axis=1) @ G0[1].reshape(G0[1].shape[0], -1)).reshape(N[1], -1) @ G0[2].reshape(G0[2].shape[0], -1)).reshape(N[1]*N[2], -1) @ (np.squeeze(G0[3][:, np.array([36]), :], axis=1) @ np.squeeze(G0[4][:, np.array([54])], axis=1))).reshape(N[0],N[1])


plt.rcParams.update({'font.size': 16})                        
Xx1, Xx2 = np.meshgrid(Phi, Phi)
fig = plt.figure()
plt.contourf(Xx1, Xx2, P_a.T, 100)
plt.colorbar()
plt.xlabel('$ \psi_1 $')
plt.ylabel('$ \psi_2 $')

fig = plt.figure()
plt.contourf(Xx1, Xx2, P_b.T, 100)
plt.colorbar()
plt.xlabel('$ \psi_1 $')
plt.ylabel('$ \psi_2 $')

fig = plt.figure()
plt.contourf(Xx1, Xx2, P_c.T, 100)
plt.colorbar()
plt.xlabel('$ \psi_1 $')
plt.ylabel('$ \psi_2 $')

xX1, xX2 = np.meshgrid(Xp, Xp)
fig = plt.figure()
plt.contourf(xX1, xX2, P_d.T, 100)
plt.colorbar()
plt.xlabel('$ X_1 $')
plt.ylabel('$ X_2 $')

xX1, xX2 = np.meshgrid(Xp, Xp)
fig = plt.figure()
plt.contourf(xX1, xX2, P_e.T, 100)
plt.colorbar()
plt.xlabel('$ X_1 $')
plt.ylabel('$ X_3 $')

xX1, xX2 = np.meshgrid(Xp, Xp)
fig = plt.figure()
plt.contourf(xX1, xX2, P_f.T, 100)
plt.colorbar()
plt.xlabel('$ X_2 $')
plt.ylabel('$ X_3 $')

plt.show()
